In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import os

In [49]:
fanfics = os.listdir('Fanfic_HTML')

remove_fanfics = ['.DS_Store', 'New Beginnings AU.html', 'Book Commentary The Book.html', 'Google Reads Fanfics.html', 
                  'Overheated.html', 'The Internet Is Forever.html', 'Bound.html', 'The Land as Bitter as.html', 
                  'My Favorite Mistake.html', 'How Flowers Bloom In The.html', 'These Boundaries Crossed.html', 
                  'Book Commentary.html']
for item in remove_fanfics:
    fanfics.remove(item)

for index, item in enumerate(fanfics):
    fanfics[index] = 'Fanfic_HTML/' + item

In [26]:
def extractStoryRating(classTags):
    #code to extract story rating
    i = 0
    for element in classTags:
        i += 1
        if i == 4:
            story_rating = str(element.find('a').string)
        elif i > 4:
            break
            
    return story_rating

def extractArchiveWarning(classTags):
    i = 0
    for element in classTags:
        i += 1
        if i == 8:
            archiveWarning = str(element.find('a').string)
        elif i > 8:
            break
            
    return archiveWarning

def extractCategory(classTags):
    #code to extract category
    i = 0
    for element in classTags:
        i += 1
        if i == 12:
            story_category = str(element.find('a').string)
        elif i > 12:
            break

    return story_category

def extractFandoms(classTags):
    #code to extract fandoms
    i = 0
    j = 0
    fandom_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Fandom:":
            j = 2 + i
        elif i == j:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += str(obj.string) + ', '
            fandom_text = fandom_text[:-2]

    return fandom_text

def extractRelationships(classTags):
    #code to extract relationships
    i = 0
    j = 0
    story_relationships = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Relationship:":
            j = 2 + i
        elif i == j:
            raw_relationships = element.find_all('a')
            for obj in raw_relationships:
                story_relationships += str(obj.string) + ', '
            story_relationships = story_relationships[:-2]
            
    return story_relationships

def extractCharacters(classTags):
    #code to extract characters
    i = 0
    j = 0
    characters = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Character:":
            j = 2 + i
        elif i == j:
            raw_characters = element.find_all('a')
            for obj in raw_characters:
                characters += str(obj.string) + ', '
            characters = characters[:-2]
            
    return characters

def extractAddTags(classTags):
    #code to extract tags
    i = 0
    j = 0
    additionalTags_text = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Additional Tags:":
            j = 2 + i
        elif i == j:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += str(obj.string) + ', '
            additionalTags_text = additionalTags_text[:-2]

    return additionalTags_text

def extractSeries(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Series:":
            j = 2 + i
        elif i == j:
            series = str(element.find('a').string)

    return series

def extractCollections(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    collections = ""
    for element in classTags:
        i += 1
        if str(element.string) == "Collections:":
            j = 2 + i
        elif i == j:
            collections = str(element.find('a').string)

    return collections

def extractPubStats(classTags):
    #code to extract publication stats
    i = 0
    j = 0
    for element in classTags:
        i += 1
        if str(element.string) == "Stats:":
            j = 2 + i
        elif i == j:
            pub_stat = str(element.string).strip()
            stat_length = len(pub_stat)
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[stat_length-4:stat_length+1]
            if " " not in story_word_count:
                continue
            else: 
                story_word_count = story_word_count[1:]
    
    return story_publish_date, story_word_count

In [27]:
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = np.nan
    story_character = np.nan
    relationship = np.nan
    addTags = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    pubDate = np.nan
    series = np.nan
    collections = np.nan
    wordCount = np.nan
    tagsClass = soup_obj.find(class_="tags")
    for element in tagsClass:
        if str(element.string) == "Rating:":
            rating = extractStoryRating(tagsClass)
            
        elif str(element.string) == "Archive Warning:":
            archiveWarning = extractArchiveWarning(tagsClass)
            
        elif str(element.string) == "Category:":
            category = extractCategory(tagsClass)
            
        elif str(element.string) == "Fandom:":
            fandom_text = extractFandoms(tagsClass)
        
        elif str(element.string) == "Relationship:":
            relationship = extractRelationships(tagsClass)
            
        elif str(element.string) == "Character:":
            story_character = extractCharacters(tagsClass)
            
        elif str(element.string) == "Additional Tags:":
            addTags = extractAddTags(tagsClass)
            
        elif str(element.string) == "Series:":
            series = extractSeries(tagsClass)
            
        elif str(element.string) == "Collections:":
            collections = extractCollections(tagsClass)
            
        elif str(element.string) == "Stats:":
            pubDate, wordCount = extractPubStats(tagsClass)
            
        i += 1
    
    local_tag_dict = {'rating' : rating,
                'archiveWarnings': archiveWarning,
                'category' : category,
                'fandom' : fandom_text,
                'relationships' : relationship,
                'character': story_character,
                'additionalTags': addTags,  
                'series': series, 
                'collections': collections,
                'pub_date' : pubDate, 
                'word_count' : wordCount
               }
        
    return local_tag_dict

In [28]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        author = str(info).split('-')[1]
        author = author.strip()
    return author

def extractTitle(soup_obj):
    #code to extract title
    i = 0
    for element in soup_obj.find(class_="message"):
        if i == 1:
            title = element.string
        elif i > 1:
            break
        i += 1
        
    return title
    
def extractStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 3:
            if "<span>" in str(soup_obj):
                story_spantag_text = element.find_all('span')
                for obj in story_spantag_text:
                    if obj.string == None:
                        continue
                    else:
                        story_text += str(obj.string) + " "
                        
            elif "<div>" in str(soup_obj):
                story_divtag_text = element.find('div')
                story_ptag_text = story_divtag_text.find_all('p')
                for obj in story_ptag_text:
                    features = obj.contents
                    for feature in features:
                        if "br" in str(feature):
                            continue
                        else:
                            story_text += str(feature) + " " 
                            story_text = story_text.replace("<i>", "")
                            story_text = story_text.replace("</i>", "")
                        
            else:
                story_ptag_text = element.find_all('p')
                if len(story_ptag_text) == 1:
                    story_text = str(element.find('p'))
                    story_text = story_text.replace("<p>", "")
                    story_text = story_text.replace("</p>", "")
                    story_text = story_text.replace("<br/>", "")
                else:
                    for obj in story_ptag_text:
                        if obj.string == None:
                            continue
                        else:
                            story_text += str(obj.string) + " "
                       
        elif i > 3:
            break
            
        i += 1
        
    return story_text


def extractStoryChapters(soup_obj):
    #code to extract title and story text
    i = 0
    j = 0
    story_chapter_text = ""
    chapters = soup_obj.find(id="chapters")
    for element in soup_obj.find(id="chapters"):
        i += 1
        if str(element.string) == "chapter content":
            j = 2 + i
        elif str(element.string) == "/chapter content":
            j = 2 + i
        elif i == j:
            if 'blockquote class="userstuff"' in str(chapters.find(class_="userstuff")):
                k = 0
                m = 0
                for element in chapters:
                    k += 1
                    if str(element.string) == "chapter content":
                        m = 2 + k
                    elif k == m:
                        chapter_ptag_text = element.find_all('p')
                        
            elif "<span>" in str(chapters.find(class_="userstuff")):
                n = 0
                o = 0
                for element in chapters:
                    n += 1
                    if str(element.string) == "chapter content":
                        o = 2 + n
                    elif n == o:
                        chapter_ptag_text = element.find_all('span')
                        
            else:
                userStuff = chapters.find(class_="userstuff")
                chapter_ptag_text = userStuff.find_all('p')
                
            for obj in chapter_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_chapter_text += str(obj.string) + " "
                    
    return story_chapter_text

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        tag_dict = extractTags(soup)
        author = extractAuthor(soup)
        title = extractTitle(soup)
        if "chapter content" in str(soup):
            story_text = extractStoryChapters(soup)
        else:
            story_text = extractStory(soup)
        
        story_dict = {'title': title,
                      'author': author,
                      'story_text': story_text, 
                      'url': url,
                     }
        story_dict.update(tag_dict)
        
        ao3_dict[i] = story_dict
        i += 1
    return ao3_dict

In [29]:
ao3_dictionary = createAO3dict(fanfics)

In [30]:
ao3_df = pd.DataFrame.from_dict(ao3_dictionary, orient="index")
ao3_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,NaN,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",Genderbending,NaN,Yuletide 2011,2011-12-22,1071
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,NaN,NaN,NaN,NaN,NaN,2019-06-15,750
2,One Recurse,orphan_account,No one of them could be Cindi Mayweather. It's...,http://archiveofourown.org/works/333825,Mature,No Archive Warnings Apply,F/F,"Metropolis: The Chase Suite - Janelle Monáe, T...",NaN,"Alpha 9000s, Cindi Mayweather","Androids, Clones, Character of Color, Porn Battle",NaN,Porn Battle XIII (Lucky Thirteen),2012-02-06,889
3,"ask, and ye shall receive",Ariaste,"He feels rather daring about it, especially wh...",http://archiveofourown.org/works/19964368,General Audiences,No Archive Warnings Apply,M/M,"Good Omens (TV), Good Omens - Neil Gaiman & Te...",Aziraphale/Crowley (Good Omens),NaN,"a tiny accidental fic, cute and sweet, puns",NaN,My Favorite Ineffable Husband Fics,2019-07-24,441
4,In Gardens,Bitterblue,"Neither of them is sure how to start, but Bit...",http://archiveofourown.org/works/432750,General Audiences,No Archive Warnings Apply,Gen,"Bitterblue - Kristin Cashore, Seven Kingdoms T...",NaN,"Hava (Bitterblue), Bitterblue (Graceling)",NaN,NaN,NaN,2012-06-13,279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1037,Exchange,swiddershins,"He’d inquired about the coat’s price, carefull...",http://archiveofourown.org/works/14392434,General Audiences,No Archive Warnings Apply,Gen,"Neverwhere - Neil Gaiman, Neverwhere - All Med...",NaN,Marquis de Carabas (Neverwhere),"baby marquis, baby baby baby marquis, Characte...",NaN,NaN,2018-04-22,759
1038,Peeta Mellark: In it to Survive,SilverTonguedWriter,The games were tomorrow. I had no fall back pl...,http://archiveofourown.org/works/1000334,Mature,Choose Not To Use Archive Warnings,M/M,Hunger Games Trilogy - Suzanne Collins,Cato/Peeta Mellark,"Cato (Hunger Games), Peeta Mellark","One Shot, Sex, Blow Jobs",NaN,NaN,2013-10-11,444
1039,Keeping Promises,fourredfruits,Everything happened so fast it seemed like the...,http://archiveofourown.org/works/18372194,General Audiences,Choose Not To Use Archive Warnings,F/M,"The Umbrella Academy (TV), The Umbrella Academ...","Number Five | The Boy & Vanya Hargreeves, Numb...","Number Five | The Boy (Umbrella Academy), Vany...",NaN,NaN,NaN,2019-04-06,808
1040,Interlude: It grows back,kaleigh,,http://archiveofourown.org/works/21219572,Not Rated,No Archive Warnings Apply,F/F,"Addams Family - All Media Types, Addams Family...",Wednesday Addams/Original Female Character(s),"Wednesday Addams, Pubert Addams, Lurch (Addams...","Haircuts, Sibling Bonding",Love is a four letter word,NaN,2019-10-29,959


In [31]:
ao3_df.word_count.astype('int32')

0       1071
1        750
2        889
3        441
4        279
        ... 
1037     759
1038     444
1039     808
1040     959
1041     100
Name: word_count, Length: 1042, dtype: int32

In [32]:
ao3_df.sort_values(by=['word_count'])

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
880,Dreamer,Tsukiakari1203,Kamala starts out ordinary She writes stories...,http://archiveofourown.org/works/28513917,General Audiences,No Archive Warnings Apply,NaN,"Ms. Marvel (Comics), Champions (Comics)",NaN,Kamala Khan,NaN,Marvel Poetry,NaN,2021-01-02,41
340,Baby Names,katharhino,"""Let Calcifer name it,"" he said sleepily, look...",http://archiveofourown.org/works/723619,General Audiences,No Archive Warnings Apply,F/M,Howl's Moving Castle - All Media Types,Sophie Hatter/Howl Pendragon,"Howl Pendragon, Sophie Hatter",3 Sentence Fiction,NaN,NaN,2013-03-17,77
1013,The Letter,kaleigh,"Dear Mother and Father, PS - If you arrive and...",http://archiveofourown.org/works/9842627,Not Rated,Choose Not To Use Archive Warnings,F/F,"Addams Family - All Media Types, The Addams Fa...",Wednesday Addams/Original Female Character(s),"Wednesday Addams, Morticia Addams, Gomez Addams","Letters, Alternate Universe - College/University",Love is a four letter word,NaN,2017-02-20,77
739,Day 4 - Warmth,Warmth,Jake and Raleigh take up an absurdly large are...,http://archiveofourown.org/works/23785684,General Audiences,No Archive Warnings Apply,F/M,Pacific Rim (Movies),"Raleigh Becket/Mako Mori, Mako Mori & Jake Pen...","Mako Mori, Raleigh Becket, Jake Pentecost, Ama...","Mako Mori Lives, Raleigh Becket Lives, uprisin...",Mako Mori Appreciation Week 2020,NaN,2020-04-22,86
426,Family Re-forged,forged,You are learning too fast for me to be assured...,http://archiveofourown.org/works/13506459,General Audiences,No Archive Warnings Apply,Multi,The Broken Earth Series - N. K. Jemisin,Alabaster/Damaya | Essun | Syenite/Innon,"Hoa - Character, Alabaster (Broken Earth), Dam...",Drabble Day 2018,NaN,NaN,2018-01-28,99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,Airplanes Like Shooting Stars - A Wish,fresne,The black hawk perched on the buzzing power li...,http://archiveofourown.org/works/197613,General Audiences,No Archive Warnings Apply,M/M,"Supernatural, American Gods - Neil Gaiman",Dean/Castiel,"Horus (Egyptian Mythology), Inanna | Ishtar",NaN,NaN,NaN,2011-05-10,992
479,once more with feeling,Theskee,"\n I can’t stand him. No really, I pro...",http://archiveofourown.org/works/11113725,Mature,Choose Not To Use Archive Warnings,M/M,"American Gods - Neil Gaiman, American Gods (TV)",Mad Sweeney/Shadow Moon,"Mad Sweeney, Shadow Moon","Sweeney's POV, Slightly Smutty",NaN,NaN,2017-06-06,995
1007,Blood,draig_asec,Diego laughed lowly as he walked out of th...,http://archiveofourown.org/works/17940656,Not Rated,No Archive Warnings Apply,Gen,"The Umbrella Academy (TV), The Umbrella Academ...",Diego Hargreeves/Klaus Hargreeves,"Klaus Hargreeves, Diego Hargreeves","this wasn’t meant to be so, gay?, but it’s gay...",Umbrella Academy fics based off of or quoting ...,NaN,2019-02-27,995
906,In Which A Wizard Is Injured At A Rugby Match,bobblemonkey2,‘Owwwww!’ Howl stomped in through the front do...,http://archiveofourown.org/works/24504826,General Audiences,No Archive Warnings Apply,F/M,"Howl's Moving Castle - All Media Types, Howl S...",Sophie Hatter/Howl Pendragon,"Howl Pendragon, Sophie Hatter, Calcifer (Howl ...","One Shot, Lemon, bookverse, Established Relati...",NaN,NaN,2020-06-02,999


In [50]:
#grab all the blank story_text
blankStory_df = ao3_df[ao3_df["story_text"] == '']
#ao3_df = ao3_df.drop(blankStory_df.index)
#ao3_df[ao3_df["story_text"] == '']

In [46]:
ao3_df.story_text[95] = "What’s in a Name I need something of yours, Sophie. … How about your eyes? Fire demons are not benevolent entities. Calcifer’s fiery shell concealed a mind of marble and a will of iron, and when Sophie said hesitantly, ‘Would just one be enough?’ he seized her right eye before she could change her mind. He made sure to cauterise the wound – wouldn’t do for her to die before she’d made good on her promise – and then he looked out at the world through her eye. When he saw that she loved Howl more than she loved life itself, he knew he’d get to feast on her again and again before this war was over. A Dog’s Life His name is Heen. He’s my errand dog. Heen had been a man once, and a wizard. He’d fallen for Madame Suliman, and made the mistake of letting her see his devotion. ‘At heart you’re a little lapdog,’ she said, and he was. That was years ago. At least he thought it was years. Basset hounds had trouble thinking beyond suppertime, but he’d long since stopped yearning to chase the rabbits that nibbled the palace lawns. When his mistress ordered him to dog the old woman back to the wizard’s lair he’d gone gladly. He hoped, as he reported back, he might have earned her respect at last. This Dumb War You should go home anyway and tell your king to stop this dumb war. Prince Justin was a poet, not a soldier or a spy. He didn’t make it back across the border. When he heard the news, his father ordered an invasion. Crops were burned, fields salted, cattle slaughtered, wells poisoned, roads mined, bridges blown up and countless lives squandered, till in the end his armies encircled Ingary’s capital. The king ordered Madame Suliman to retaliate. The magic that’d been drained from the Witch of the Waste, the Wizard Pendragon, and a dozen other deserters, was loosed all at once over the enemy’s capital. Afterwards not one stone stood on top of another."
ao3_df.story_text[207] = "Step 1: Gather your ingredients. Slip into your husband's shop and nick some of his supplies; grin as he presses a kiss into your face, getting flour in your hair. Remember a time when you would have killed for that flour, would have killed for anything to eat, and have your smile falter a second. Regain it when he traces your nose with a bit of flour, then kisses it. Smile as you watch your child \"help\" knead dough. Go back to the kitchen. Step 2: Preheat the oven. Remember how hot Peeta's forehead was when you found him in your first games. Feel your heart stop, then shake it off. Butter a pan and remember, distantly, that your sister loved pound cake. Step 3: Cream the butter and sugar. Think of your mother's hands at work in the kitchen as you blend the ingredients; remember how often your stomach went hungry. Feel nauseous. Channel your energy into beating the butter and sugar into a rich, thick cream. Step 4: Add the eggs. Hear the front door open and tense; feel your heartbeat beat frantically for a second before you remember that you're safe. Feel your husband's sturdy arms wrap around your own as he asks how it's going; feel the warmth of his grin as you tell him you're making a pound cake. Feel his fingers linger on yours as he hands you the vanilla. Pour in a bit more than is perhaps necessary, then give it back to him. Step 5: Have your husband give you the flour. Pour it exactly into the cup; smile as your husband insists you have a baker's knack for measurements. Don't tell him that you're too used to having to make the first attempt count because supplies were so scarce. Step 6: Pour the batter into the pan. Smile as Peeta runs his finger along the edge and tastes the runoff. Flush with pride as he pronounces it as good as his mother's. Wish, not the first time, that she was here with you and not in the valley of the dead below. Step 7: Feel the comforting warmth of your husband vanish as you press the pan into the oven. Feel a soft flutter as he promises to come back later. Watch him grip his hand on the door a bit too long, and know he's suffering a flashback. Wait until he blinks, then take his hand and walk back with him to the shop. Stay until he comes back to you. Step 8: Cross back to your modest homestead. Pull out your cake. Feed your daughter a slice and marvel at how bright her eyes are; the blue color is the same as her father's eyes, but these eyes are still shining, have never known her father's pain. Smile as she reaches up to hug you. Press kisses to her head and hold her tight, remembering all that you and Peeta have lost, but also what you have gained. Step 9: Surprise your husband with a slice of cake as he closes up the shop. Laugh when he asks what the occasion is. Laugh again when he calls himself the luckiest man, even if you both know it isn't true. It's true enough. Step 10: Have a bit yourself as you come home again; the vanilla tastes sweet on your tongue. Think of the taste as it clings to your mouth, as you put your daughter to bed, as you climb into your exhausted husband's arms. When he asks you what's wrong, blurt out that you want another baby, and listen to his stunned silence as you realize it's true. He presses a kiss to you, thankful, and you taste the sweet vanilla scent on your tongue as his weight shifts over you, and you close your eyes and remember that you have survived, that one day, your children, too, will learn to hold sweetness on their tongue. It is not perfection, but it is enough."
ao3_df.story_text[263] = "The air was bloated with darkness. But even in the darkness the body could be seen. It was a Jew's, with one crooked finger, dark drifts of hair, an almost random birthmark. He had been sloshed out onto the floor, where he now lay sprawled on his back, arms tangled like a black-haired, brown-eyed swastika. His name, if names ever really mattered, was Max Vandenburg. He heard something click far away. Light whispered through the room that always smelled of him.\"Hey Maxi, I got you some stuff\", a man was somehow standing in front of him. For a second the blue eyes and blonde hair made him want to scratch out his own eyes. But then he remembered, as he always did, that this was Walter. Walter Kugler, the best fist fighter in Stuttgart, the best, most perfect-looking Aryan, the best friend a Jew could have. \"Here\", paper crinkled between them, \"it's a lot better now that I've gotten in.\" Max hummed. It was better, he thought. But he wasn't sure if it tasted better because of the quality, or simply because he hadn't eaten in so many, many days. He was just chomping down a knot of bread when he stopped. And his mind stretched to include the room, the darkness and Walter. \"Gotten into what?\" crumbs spooled onto the floor. Max didn't look but he knew the dark would swallow them. Walter riffled a hand through his hair. \"Oh, you know, Maxi. The usual-\" \"The army. You're in the army\", Max mumbled. Walter leaned closer. \"I had to. They're getting too close...it's better...this way\", he let the darkness take up the dialogue. The Jew said nothing. His hands were talking though, his eyes, the rim of his lips. Finally: \"I have to get going, Maxi. I'll...be seeing you\", Walter stood up but he didn't leave. His hand, his Nazi hand, reached out and found, past the seconds, past the heartbeats, a pale swathe of Jewish cheekbone. And the music of his nails, the filmy twinkling of a thumb print, made Max smile. Darkness whisked through the spaces between his teeth. He quickly stopped. Walter didn't. Walter pressed his forehead against Max's, his Aryan hair winding around Jew-black.He pasted his blue, blue eyes onto Max's brown; so close they could see the eyes inside their eyes. And even though there was no earthly reason for it, Max traced with his tongue; \"It's okay Walter. Hitler's okay\", onto the dark. Somehow, in some strange way, there were men behind this man, their blood; red-white-black, flowing in his veins too. And somehow, in some strange way, a Nazi could love a Jew. And be loved right back. All of him. By a man named Max Vandenburg."
ao3_df.story_text[295] = "Chapter 1 Everybody knows that anniversaries are important. Curtis certainly did. That’s why he had taken the time after school to buy flowers, real ones from a florist. He’d had trouble with finding a bouquet. He didn’t just want to ask his grandmother to give him flowers from her garden, Bri would recognize them and he wanted her to know he cared enough to spend money on her. Shopping in the uptown neighborhood where they went to school provided less than thrilling options of flowers, none of them really felt like Bri. The flowers he could find in the Garden might as well have been bought in a grocery store. It looked about like he would be giving up and buying Bri thoughtless chocolates, but his grandmother, busybody that she was, could tell something was wrong and would not rest until she pried it from him. He was glad she did. She ended up giving him the information of a Garden Heights florist who she was friends with. The selection was amazing, it ended up being larger than both his failed attempts, and within his price range too. He’d ordered a bouquet of black lilies, known colloquially as monkey tails, and seeplakkie. The bouquet was small, he was splurging, but not made of money after all. But Bri wouldn’t mind, she wasn’t greedy like that, and as far as he knew she’d never been given flowers before. He thought she might like a candy bar bouquet but nixed the idea after recalling her lingering tenderness over the drug dealer incident. She would love the flowers, hopefully. After a year of dating, flowers alone could not do justice to the B-R-I-liiant Bri. It was time of course to get the flowers for Bri; they were already bought and paid for; they just needed to be picked up. He pushed open the door and saw some familiar black braids on a familiar girl at the counter also picking up flowers. “Bri?” She turned around. “Curtis?” The boy at the counter passed over the flowers to Brianna. “Thanks Robert,” she said politely. “Are those for me?” he asked, flattered that she’d thought of him. “Oh. Um, they’re for Aunt Pooh.” And suddenly Curtis felt like an idiot. How could he have forgotten that their anniversary was also the day her aunt got arrested? What kind of boyfriend could forget that?” “Oh?” “I got your Nikes cleaned by this girl in the Garden, Starr, cause they were gettin’ kind of stanky.” “So that’s where they went.” “They’re at my place right now, I was going to give them to you at school, but if you want them back now—” “Bri. It’s okay. Don’t worry about it.” Curtis approached the counter. “I’m here to pick up a bouquet of seeplakie and black stick lilies for my girlfriend.” “Oh Curtis,” Bri mumbled. “I’ll be right back with those sir,” the shop boy said before disappearing into a back room. “You going to visit Pooh today?” “Yeah. My mom’s just outside, she didn’t want to come in the store.” In a fit of impulsiveness he asked, “Can I come with?” Bri hesitated for a moment before saying, “If that’s what you want.” Robert returned with the flowers and said, “It looks like you already paid online. Have a nice day sir.” “You too,” Curtis spoke absently as he left the florist’s with Bri. Curtis held his flowers out to Brianna. “These are for you.” “Thanks Curtis. I’d say give them to my aunt, but I don’t think she needs two whole bouquets.” She sniffs the bouquet. “These are amazing Curtis. I love you so much.” Bri gave him a kiss which was interrupted by her mom honking her car horn. They both chuckled and climbed in the car. Bri riding shotgun and curtis in the back. “Curtis what you doing here?” Mrs. Lawson asked. “I’m coming to meet Aunt Pooh.” “You good with this Bri?” her mother asked. “If you are.” “Alright Curtis,” Mrs. Lawson said. “Buckle up.” Chapter 2 Phones had to stay in the car, so did purses, and Curtis left his belt just in case. The flowers had to be brought to staff, and were announced ahead of time. They couldn’t even give Pooh the flowers they’d bought themselves, the staff would do it apparently. Mrs. Jackson seemed to be side-eyeing him like she couldn’t understand what had motivated him to drive up to visit someone else’s family in prison; she was also side-eyeing Bri because she knew what Curtis’ motivation was. “You ready Mom?” Bri asked. Curtis was pleased to see her referring to Mrs. Jackson as “Mom” instead of “Jay”, their relationship had developed so much; Brianna was working on her trust-issues, especially in relation to her mother.” “I am if you are baby.” “Of course,” she sniffs, “we didn’t drive up here for nothin’. It’s a shame that Lena couldn’t join us.” “Lena has to work today. She’s such an upstart.” “Aunt Pooh chose well with her.” “She certainly did.” The trio entered the building and passed through security. Security was as much of a nightmare as Curtis had remembered it to be. He didn’t like the way the guards looked at him, like he wasn’t good enough, like he was a threat. It made it worse to be here with his girlfriend, knowing that she was not only good enough, but the best of them. She wasn’t a threat either; she could stand up for herself, but only if you threatened her first. Curtis loved that Brianna didn’t allow herself to be anybody’s doormat, even though people gave her shit for being a “stereotypical hood-rat”, those people don’t know her or what they were talking about. In Curtis’ opinion, anybody who didn’t admire Bri for her authentic and courageous self, had their head up their ass. So of course the guards looking at her and her mom like they weren’t shit pissed him off. That wasn’t right and it wasn’t at all fair. The whole situation was unfair, but if Curtis dwelled on it, he wouldn’t be able to meet Pooh with a cheerful attitude, or any attitude but a morbid one. Getting to Pooh took some time and borderline humiliation, but they did get to see her, which was what really mattered. They met her in what kind of looked like a cafeteria, a room full of tables where inmates sat with their families and other visitors. “Bri! Jayda!” Pooh greeted excitedly. She looked at Curtis, “Who’s your friend?” “Aunt Pooh, this is Curtis Brinkley. He’s… my boyfriend.” Bri took Curtis’ hand. “Boyfriend? You didn’t tell me about no boyfriend in your letters.” Bri wrote her aunt letters; how sweet. I should write my moms letters too. “I wanted to tell you about Curtis in person Aunty.” Pooh looked at Mrs. Jackson. “He takin’ good care of her, Jayda?” “Curtis is a fine young man Katricia.” “Katricia? You going’ for a formal greeting Jayda.” “Formal? With you?” Mrs. Jackson reached out like she wanted to touch her sister but stopped herself. Touching inmates wasn’t allowed. The whole party grimaced at the aborted movement.“How you been Pooh?” Mrs. Jackson asked tenderly.“Can’t complain. How ‘bout you? Lena? Scrap? How’s the Garden lookin’.”“All good. I found a job, part-time, but it’s good to be workin’.”Pooh forced herself to smile. “That is good.”“Lena misses you. Does she visit?” Bri asked.“She does as much as she can, but it’s hard for her to get up here often. She writes. She puts money in my commissary account. Don’t you worry Bri, your Aunt Lena still thinks of me.”“Aunt Lena? Since when has she been Aunt Lena.”“Well it’s not official, but we’ve been talking about having a courthouse wedding once I get out.”“That’s great!” Bri and Mrs. Jackson said in unison.“I’m happy for you Pooh,” Mrs. Jackson said.“Is Lena allowed to know we know?” Curtis asked.“You know Lena?”“Course I do. What kind of boyfriend doesn’t know the family?”“I don’t know.” She looks at Bri. “What kind of aunt doesn’t know about the boyfriend?” “Sorry Aunt Pooh,” Bri apologized sheepishly. “Well I’ve heard all about you Ms. Bordeaux. Bri always talkin’ bout how cool and supportive you are.”“Ms. Bordeaux? Bri you got one with manners.” Bri smiled at Pooh’s words. “I’m glad to know that Bri talks about me, and you can call me Pooh, everyone does.” “I am thrilled to meet you Pooh. This whole family is just amazing.”The women at the table smiled, and Curtis knew he’d done well. Bri took his hand and launched into family conversation with her aunt as though the guards, other inmates, and their families didn’t exist. Curtis was content to just watch her speak and be happy."
ao3_df.story_text[309] = "Asher ‘He was sitting really close to me. I could feel his warmth. He looked at me, and smiled. It made me melt. “Asher…” He cooed. “Asher, I love you…” He whispered to him. I smiled. “I love you too, Jonas…” I heard myself say. He grinned, and leaned into me. He pressed his lips to mine, and I pressed back. He tasted so sweet.’ Asher woke up in a cold sweat. “W-what… What was that?” He mumbled, looking down at the pale sheets. To make it even more weird, it was Jonas. His best friend. Jonas‘”Asher!” I screamed. I could feel him, gripping my hips. He moaned, thrusting hard into my lower regions. This was breaking so many rules. “Jonas…” He gasped out between thrusts. “Ugh! Harder!” I heard myself whimper. He complied, hitting something down there that made me see stars. I came, spilling my seed on my chest. He thrusted a few more times, before groaning and climaxing inside me. “I love you, Jonas.” He mumbled into my ear. “I love you too, Asher…” ‘ Jonas woke up with a jerk, eyes darting down the the wetness near his lower regions. He made a mess of his sheets. ‘Is it sad that I want that to be a reality?’ He thought to himself as he changed his bedding. Fiona ‘ I scooted closer to her. She was so pretty, I couldn’t stand it. Her long curly hair, pale eyes, were so alluring. She was glowing, so full of life. I pressed my lips to her cheek, it tasted like cherries. It stuck to my lips. She giggled, putting her arms around my neck. “So beautiful.” I said, praising her. She pressed her lips my forehead, mumbling, “I love you, Fiona.” I smiled. “I love you too, Rosemary.” ‘ Fiona yawned, batting her eyes open from such a lovely dream. Rosemary ‘She touched me. She put her hands on my breasts, and rubbed them. It felt so good. I heard myself gasp, and lean into her gentle touch. “Fiona~!” I mewled. She kissed my neck in such a delicious way. I ran my hands through her long red hair. It was so gorgeous, I loved it so much. She moved south, taking my skirt off. “You look so tasty, Rosemary.” She said, licking her lips. I could feel my thighs tremble. She was so close to my private area that I needed the most attention to. “I love you, Fiona.” “Mhmm…” ‘ Rosemary gripped her sheets, opening her eyes. ‘That was a highly pleasant dream’, she thought."
ao3_df.story_text[371] = "There are certain rules to international politics that Hanna must respect, because she is a princess and the commander of an army. Most of the rules don't give her trouble: don't invade needlessly, don't mercilessly crush anyone, don't shoot your own people. Being commander of the Dellian army involves a great deal more public works and horse training than fighting, and she likes it that way. But when she sees the young Queen of Monsea for the first time, Hanna deeply regrets don't get romantically involved, because as sensible as that advice may be, it is going to make her stay in Bitterblue City very uncomfortable."
ao3_df.story_text[416] = "Patrick and Brad talk after the cafeteria incident. Lyrics from 'The Scientist' by Coldplay. Come up to meet you, tell you I'm sorry You don't know how lovely you are... The cuts on Brad's face were healing, the numerous bruises slightly faded, when Patrick arrived at his locker to find his once-boyfriend standing there. 'I'm surprised you're going to taunt me without backup this time,' Patrick told him with an attempt at indifference. 'Please move aside?' Brad seemed about to speak, but unable to find words. ...I had to find you Tell you I need you Tell you I set you apart... Brad took a breath, then sighed nervously, and didn't move. 'Not taunting,' he mumbled eventually. Patrick smiled weakly. 'Oh, good. In that case, you're not here for anything, so I'd thank you to move. There are things I'd rather be doing.' 'I need to talk to you,' Brad burst out. 'Please, Patrick.' ...Tell me your secrets And ask me your questions Oh, let's go back to the start... 'I'm not interested in your shit,' Patrick's voice was almost as certain-sounding as he wanted it to be. ...Running in circles Coming up tails Heads on a science apart... Brad faltered. 'It's not going to be- listen, OK? God, I didn't want to do this in public... I'm sorry. And I really mean it. I was disrespectful to you; that was wrong, and I'm not asking you to, because I'll never expect you'll want to, but I wish we could give it another try.' His eyes roamed over Patrick's face until Patrick looked back. Brad dropped his gaze. ...Nobody said it was easy It's such a shame for us to part Nobody said it was easy No one ever said it would be this hard Oh, take me back to the start... A small crowd had gathered over the course of Brad's speech. Patrick noticed them staring. 'Wanna go somewhere else?' he murmured, quietly enough that only the apologetic figure before him could hear. Brad nodded gratefully, and Patrick led him to an out-of-the-way dead-end hallway. 'Now tell me what you need someone to hear,' Patrick requested soberly but gently. ...I was just guessing At numbers and figures Pulling the puzzles apart... 'That day... you know the one, the cafeteria one... I didn't know how to feel... I didn't want anyone to know... My father said some stuff... more, after you'd gone. Not just the 'unnatural' stuff, the 'how are you going to succeed in anything' stuff. He went, he went really quiet. Started saying how disappointed and ashamed he was.' ...Questions of science Science and progress Do not speak as loud as my heart... Brad swallowed. 'Anyway, I know he's the shittiest- you know what I mean... but I didn't want to be a disappointment, so I thought... but it didn't work. I still hated it when they hit you. But I didn't do anything about it. I still didn't want to be a disappointment. And I kind of wanted to keep my friends as well. But they...' he sighed. 'You like (sorry, liked) me as whoever I, kind of, am, you know (this is such cliched crap), like, by myself. As myself. If they knew... they'd treat me like I treated you... They're not... I like you better.' ...Tell me you love me Come back and haunt me Oh, and I rush to the start... Patrick bit his lip. 'You'll be in big trouble with your father if he finds out you spoke to me like that.' Brad nodded. 'I need a chance to think, Brad. Can you give me that? Like, a day? I'm angry, a bit angry, with you, but I don't want to get you in trouble. And I think I'd like to... I just need time to think, OK?' Brad nodded again. ...Running in circles Chasing our tails Coming back as we are... 'I, er, we should probably get on,' said Patrick. 'I'll talk to you, OK?' He touched Brad's hand briefly and smiled a quick smile and started away. ...Nobody said it was easy Oh, it's such a shame for us to part Nobody said it was easy No one ever said it would be so hard... Patrick turned back. 'Brad? I hope I didn't hurt you too much, when I,' he laughed guiltily, 'punched you.'Brad smiled. 'It's fine.'...I'm going back to the start"
ao3_df.story_text[437] = "“This isn’ what I thought my life’d be like,” Arthur slurred, staring up at the ceiling of his cabin with unfocused eyes. He was sprawled across the bed, his clothes from the waist up discarded over the back of a nearby chair. Ford, sitting beside him on the edge of the bed, dabbed some vile-smelling ointment onto the bruises and scrapes on Arthur’s bared chest. It was harder to find a clear, healthy patch of skin than a damaged one. “Is that so.” Arthur blinked, a little lopsidedly because his left eye was somewhere in the process of swelling shut. “Yeah,” he said, “it is. I wan’ed… I dunno, something simple. Tradish’nal. ’M… not very intereshting, y’know,” he hiccoughed dully. “So you decided to try your hand at starting a drunken bar fight?” The Betelgeusian snorted. “That’s traditional. I saw it in a black and white film once.” He worked the ointment into a particularly deep scrape on Arthur’s shoulder. “It didn’t end well, in case you were wondering. The main character didn’t seem to realize that those sorts of things are supposed to be good distractions for slipping out unnoticed without paying your tab, not opportunities to stay and get mashed to a pulp.” “I thought,” Arthur continued, “maybe I’d live out in the country a few more years, finda nice girl to settle down with, an’ when all’s said’n done I’d… be happy…” He laughed drunkenly, caught halfway between irony and derisive mirth. “But it din’t turn out that way, did it Ford?” Ford rolled his eyes. “No it zarking well didn’t,” he grumbled, because part of the irony of the situation was that it had been his idea to get Arthur so drunk in the first place. “I could still be out having a good time if it weren’t for—” “Listen t’me, Ford.” Arthur brought his hand up and caught Ford’s wrist, tugging on it insistently. “Listen t’me… That… whateverwas, alien I got inna fight with, did it ‘cause he was insulting you. You din’t hear, but he was being really awful an’ you din’t mean to knock over his drink but he wouldn’t listen. So I hit him,” he finished proudly. A look of utter disbelief crept across Ford’s face. “That’s it?” he demanded, trying to work out why anyone would want to be beaten up on someone else’s account. Like sarcasm and apologies, it was a difficult concept for him to grasp. Arthur gazed drowsily up at him. “For zark’s sake, why?” “’Cause ’s—” Arthur yawned “—all s’big out here an’ I had these dreams that I can’t do anything with now… ’s all far ‘way… insig, insignif’cant… I din’t need to see the world, much less th’whole Universe, but you… There’s you… an’ I love you, Ford. M’kay?” Ford stared at him, utterly bemused. “You do?” “’Course.” Arthur smiled shyly, then yawned again. “’M tired…” “Oh.” Ford considered for a moment. If Arthur were even the tiniest bit sober, he knew, they would not be having this conversation. Ordinarily the human seemed to understand that the best way not to become terribly depressed about the destruction of his home planet was to not bring up what his life would be like if he were still on it or what his plans would have been had it continued to exist. Ford wondered if Arthur was finally beginning to let go of the small blue-green planet and adjust to life as a hitchhiker. Curious, he shifted in Arthur’s loose grip and experimented with holding his hand. The smile on Arthur’s face relaxed from shy into something warmer and unguarded, and Ford grinned back because that was exactly the sort of unperturbed peacefulness that Arthur had been lacking amidst all his panicking. And, alcohol or not, it was because of Ford. “You are so drunk,” Ford pointed out, because he didn’t think that Arthur was in much of a state to realize this for the obvious fact that it was. “But,” he added thoughtfully, “I think you actually mean that.” Arthur nodded sleepily up at him. “Cross my heart an’ hope t’… t’…” “Okay, I believe you.” He wondered how much of this Arthur would remember in the morning, if any. In all probability, Ford mused as he finished dabbing ointment on his friend’s wounds, he would end up having to explain to Arthur just why he was so sore. Ford grinned – a great deal more predatorily than Arthur, had he been awake enough to notice, would probably have been comfortable with. He was rather fond of the human himself, but he would wait until after Arthur healed from this little misadventure to try anything. “Go to sleep now.” And he watched with an unexpected degree of affection as the Earthman he’d rescued from a doomed and terminally dull planet obediently shut his eyes and drifted off."
ao3_df.story_text[441] = "At the moment of its creation, the fire spirit knew only heat and the exhilarating freedom of flight. It streaked through the heavens, plummeting towards the planet’s surface. Dimly it felt others of its kind, separate consciousnesses, sharing its fall. When they struck the earth, they flared fully to life, sending out light and heat to the uncaring night, as if declaring, “Here I am!” For many, that was the end. The falling stars struck the water’s surface and drowned, their dim shadow-bodies sinking to the bottom of the lake. The fire spirit felt its companions winking out of existence, one by one, swallowed by a cold darkness like the void where they were born. For the first time the fire knew fear and the desperate need to survive, yet it could do nothing to alter its trajectory. The fire demon struck the surface and blazed into full existence, reaching out with desperate tentacles to find fuel and air. Braced for contact with the water, the fire spirit found itself safely nestled on a dry surface and howled with joy, leaping and sending brilliant sparks up towards its elders, the distant stars. The demon was amazed to feel its joy mirrored by another consciousness. The Other was a gentle entity that sheltered the spark against its own flesh. It was more empathic communication than sound ripples that conveyed the Other’s words: “I caught you! Don’t worry, you’re safe now, little one.” The fire exhausted its core feul and reached for more, feeding off the Other’s substance. The demon felt the Other’s pain at this intrusion, but need and survival outweighed all other considerations. “More!” it demanded, “MORE!” “Here,” the Other offered, “Take what you need, friend.” A great maw swallowed the fire demon whole, suffocating it in moist, warm flesh. For an instant the demon panicked, flickering, before it encountered and merged with a very different kind of fire. Soulfire and demonfire merged, blue with orange, to forge a white-violet flame that expanded rapidly and burst from the enclosing flesh to burn freely in the air. The demon-soul fire danced high in ecstasy, fed by its new core. When it settled, it could feel the Other waiting patiently, curiosity coloring its thoughts. “Little fire, I have given you my heart. We are bound, you and I. I’ll never be alone again. Tell me, friend, what is your name?” The fire couldn’t understand all of the words, but it understood that he now shared the core of the Other’s being. It produced heat for the Other, warming and protecting it. The Other said, “Calcifer? That’s a wonderful name. My name is Howl. Someday, I’m going to be a wizard!”"
ao3_df.story_text[452] = "Chapter 1 Any minute now, Richard thought, they’re going to ask me something. Or at least remember I exist. And I’m going to smile and nod and tell them I’m fine, but I really don’t think I am. Not this time. Still the railing under his hand felt reassuring, even if he was gripping it hard enough to make his fingers ache. He turned to the man standing silently beside him, an officer he guessed although his once fine uniform was tattered and bloodstained. “What is this place?” The man only glanced at Richard for an instant before returning his attention to the Marquis and the ship's captain deep in their negotiations, but it was enough for Richard to turn back to the water unable to meet that curiously empty gaze for long. “This? This is Hell.” Richard shook his head. “No,” he said carefully, “I’ve seen Hell, I think. It wasn’t this.” Chapter 2 Contrary to popular belief, lying down didn’t actually help much with seasickness. Richard felt fairly confident making this assessment, seeing as he was simultaneously lying full length on the deck and feeling like death. He was usually okay on boats. There’d been no problems the last time he and Jessica had gone to France. Nice big ferry with a cheery café and (sweet, glorious) stabilisers. He’d managed to coax Jess briefly out on deck. He’d remembered she’d, rather uncharacteristically, worn a summery yellow scarf in her hair. She’d stayed out long enough for the roaring wind to whip it away before retreating back inside to safety. Richard hadn’t followed; instead he’d watched the scrap of silk, bright against the competing gray of the sea and sky, spiral away into the spray and screaming seagulls. He’d been the only one stupid enough to stay outside, the wind stealing his breath away even as the cold had made his hands and face burn, but he’d revelled in the moment, elated at the sense of freedom so different to anything he’d experienced on land. He groaned again as the deck lurched alarmingly under him. He was beginning to rethink his ideas on boats. Chapter 3 Richard eventually retreated below deck, ostensibly to look around but mostly to see if he felt any less useless away from the others. He was surprised by the ship. It was small. Dark and claustrophobic in places, but still cleaner and drier than he expected (though he supposed he didn’t really have much frame of reference for pirate galleons). Of course he’d not braved the lower reaches of the ship just yet, being more than content to settle by the stove and enjoy a moment of peace. He woke with a start to find the Marquis sprawled elegantly beside him, soaking up the lion’s share of the heat. The negotiations were apparently over, not that it seemed the Marquis was in a hurry to explain what had happened. Richard, knowing full well that doing anything as mundane as asking would get him nowhere, stayed silent and instead concentrated on waking up properly. After a moment the Marquis treated him to yet another of his usual cat with a canary type grins. “Seems you’re the odd man out,” was all he’d say. However it wasn’t until much later that Richard understood exactly what he meant. Chapter 4 Time didn’t obey the same rules here, but even so it had taken generations to build up the lair. But built it had been and now the rats thrived. When today’s look out ran in, chittering and squeaking with excitement, the entire colony stopped what they were doing to hear the news. The current crew had no need of food and rarely disturbed them, so any such breaks from convention were too good to miss. The look out chittered again and now they could hear the stranger begin to gingerly climb down into the hold. Almost as one they swarmed around the bottom of the ladder jostling to see their visitor. Richard paused, he still wasn’t used to the ship‘s pitching and rolling and could have sworn that the floor was moving in slow waves; he clung to the ladder, and waited for his eyes to adjust to the gloom before peering downwards again. It took his brain a little while to realise exactly what it was he was seeing, but he felt he took it rather well all things considered. He even managed a passable enough bow, no mean feat halfway up a ladder. The rats waited expectantly."
ao3_df.story_text[466] = "The Bakelite telephone on the desk rang once, then twice. It didn't make a pleasant sound to begin with, but this late -- at what some people would call an ungodly hour -- it was terribly jarring, broke the funereal silence like a sledgehammer on a church bell. Ibis sighed softly, rubbing his eyes under the round rims of his glasses; the tilting grandfather clock in the corner suggested it was a little after four. He reached for the telephone, but it shrilled off in the middle of the fourth ring. A careful murmur pushed under the closed office door, rising and falling in a way that hinted at a gruff, grunting voice. Ibis pictured Jacquel as he probably looked standing in the hallway, leaning into the telephone niche across from the bathroom, his sharp shoulders hunched and his brown skin very dark against the faded, pinstripe wallpaper. He would have both hands on the receiver, because he always held the receiver that way. Jacquel hated the telephone. Ibis couldn't imagine who was on the other end of the line, but he wasn't quite curious enough to get up and ask. He'd find out sooner or later, and he had work to do. Ibis wrote his personal histories longhand, using an expensive fountain pen older than most people still living, and a journal bound in maroon leather and finely tooled with gold. He liked the cautious, spidery lines his fountain pen made, and the crisp scritch of a sharp nib moving over good paper. He'd used a proper quill for all his writing until 1967 -- when the bank started questioning his checks and the customers started commenting with more than polite interest. The house creaked in that quiet, maudlin way it had, and Ibis frowned at the almost-empty page waiting in front of him. Children were the hardest, because there wasn't much to say; he always wanted to fill the empty spaces with everything they could have done -- should have done -- but his job only dealt with truth and facts. He pinched the bridge of his nose, setting his pen to paper. The house creaked again, a sound that felt louder and more deliberate than the first, and Ibis looked up. Jacquel was hanging in the doorway, his hand still resting on the handle. \"Good morning,\" Ibis offered. \"Not so fast,\" Jacquel said, glancing out the window. The heavy, brocade curtains were pulled back, showing a clear sky that hadn't even begun to bruise. \"You've still got another hour.\" Ibis consulted the clock again; at the tone, the time would be fifteen-past-four. \"I suppose.\" \"Why are you even up?\" \"I couldn't sleep.\" Jacquel nodded and leaned into the doorjamb. He was wearing his work clothes, a dark apron over his shirt and livid bloodstains on both his sleeves, and it occurred to Ibis that they'd been in business at this location for well over a hundred years. The people in town didn't notice because Ibis and Jacquel didn't want them to notice, but one day, they might. One day, Ibis might not be strong enough to bend the truth away from curious minds. \"Who was on the telephone?\" Ibis asked, because that felt like a safer train of thought. \"Wednesday.\" \"Really?\" They hadn't heard from Wednesday in four or five years. Before that, it had been closer to thirty. \"What does he want, after all this time?\" \"His boy is in trouble.\" \"I thought his boy was in prison.\" Jacquel shrugged lightly. \"He's out now, and evidently, he's already in hot water. Wednesday wanted to know if we'd give him a place to lie low for a few days.\" The house has been strangely quiet since Set and Horus left, and Bast was a cat more often than not these days.\"Well, we certainly have the space,\" Ibis said, pushing his pen and journal aside. \"I do hope he isn't bothered by the dead.\""
ao3_df.story_text[515] = "It was a perfectly beautiful day. Sophie took in the sickly sweet aroma of the flowers in the meadow as they gently rustled in the wind. The pink foxgloves were her favorite, and the sight of them as far as the eye could see never failed to take her breath away no matter how many times Howl insisted on coming out to the garden. It was his favorite place to be, it seemed, and he frequently whisked Sophie away to take in the sights. Today was no exception. \"Just how did you manage to get cursed by the Witch of the Waste, anyway?\" Howl asked randomly as they strolled along, breaking Sophie's train of thought. She paused and thought for a moment. \"She came into the shop one day and she said...\" Sophie cleared her throat before she attempted an impression of the witch. \"What a tacky little shop full of tacky little hats.\" She paused and stifled back a giggle. \"But the tackiest thing here by far is you.\" Howl's brow furrowed. Sophie was still giggling. \"Why are you laughing? That's not funny at all,\" Howl sniffed. \"It is now that I think back on it. I just think it was a rather dramatic thing to say.\" She smiled slightly and shook her head. \"Anyway, I marched right over to the door and showed her the way out. And that's when she cursed me.\" There was a beat of silence as Howl glanced down at her fondly.\"Well, she was wrong,\" he said. \"I think you're beautiful.\" \"You always say that,\" said Sophie as she glanced down at her shoes. \"Because it's true!\" Howl cupped her chin in his hand and lifted her gaze to his. \"You have the loveliest most beautiful features of any girl I've ever seen.\" He paused. \"And I should know. I eat their hearts,\" He said in a mockingly dramatic tone. Sophie smiled. \"Well... you're pretty lovely yourself,\" Sophie replied, not breaking his gaze. \"You think so?\" Howl breathed. Sophie leaned up to bridge the distance between them."
ao3_df.story_text[552] = "Isobel flipped the cards over for Tsukiko, on one of the rare occasions that Tsukiko came to her tent. Not a complicated reading, just three cards: past, present, and future. She was aware of Tsukiko’s eyes on her, rather than the cards. Past: six of cups, reversed. Present: the hanged man. Future: six of swords. The picture began to form in her mind. “Do you look for similarities?” said Tsukiko. “Yes, that’s part of it,” said Isobel. Ordinarily she didn’t like people asking for how she read the cards, but since Tsukiko was sort of a friend—another behind-the-scenes figure of the circus, she didn’t mind. “Sixes,” said Tsukiko. “And twelve, twice-six.” “Yes,” said Isobel. “Interpersonal support and knowledge, that’s the domain of the sixes.” Tsukiko smiled. “That’s nice. I like that.” “A girl and a boy,” said Isobel, pointing to the first card, where a young couple, surrounded by flowers, shared a kiss. “You miss him. I’m sorry.” Tsukiko snorted. “Her.” Isobel blushed. “Sorry, I didn’t—” “There’s no room for me in your cards,” said Tsukiko, gently. “I understand.” “You are too kind,” said Isobel. “Had I looked more carefully, I would have seen. The cards can only tell me the stories I already know.” “Is that a story you know?” Tsukiko, with the placid contortionist’s smile on her face, sent a shiver up Isobel’s spine. Isobel took a deep breath, and adjusted. “Did you have divinatory methods in—where you’re from?” “Not like this,” said Tsukiko, waving her arm over the cards. Isobel was suddenly, blindingly jealous of how well Tsukiko had recovered from the moment, while Isobel found herself struggling to concentrate on Tsukiko’s words, something nameless and frightening seeming to open up before her in the curve of Tsukiko’s lips and soft, dark eyes, how small her hands were. “I see,” Isobel managed. “Not so specific in the way that these are. There was a system, to do with the calendar and directions. My—friend,” she said, smiling halfheartedly and gesturing at the six of cups, “could do it beautifully. It was banned recently. Too superstitious.” “Oh,” said Isobel. “I’m sorry.” “Tell me about the rest of the cards,” said Tsukiko. Isobel told her the story of the hanged man, how the destruction of self brings new life. How it bridged the gap between the human and the divine, and it bridged the middle of the major arcana, which told a story of its own. “I don’t believe in a divine,” said Tsukiko. “That’s okay,” said Isobel. “I don’t either.” “And the last card?” asked Tsukiko. Isobel looked at the woman and the man pushing off in the boat full of swords. “A departure,” she said, finally. “You’ve been alone, and you won’t be alone any more.” Tsukiko gazed at the card. “And who will be with me?” “You ask too many questions,” said Isobel, but she gave a small smile. “I could draw another card, if you like. I don’t mind asking that question.” Tsukiko held up her hand. “I’ve taken enough of your time. You’ve other customers.” Isobel glanced to the door of her tent, where the shadows told her that a line formed outside, though she could hear no voices. If she opened the flap, she would—the tents were curious, in that way, but here, she often forgot that she and her clients were not alone. How did Tsukiko know, when she’d been facing— Isobel bowed her head. “Come back again sometime,” she said, finally. “I am curious to hear more of the story.” Tsukiko’s eyebrows raised, and she left the tent."
ao3_df.story_text[594] = "Touching is all relative. There’s a difference between touching and touching. The first time Arthur touched someone was just a few months before going off to university; a girl from another school who he’d been seeing on weekends for almost a year and still thought he would marry some day. Lying together on her bed (because the bed was the proper place for these things to happen, wasn’t it?), his shirt on the floor, her blouse just a little bit undone, her bra somehow definitely undone, and their hearts beating fast and quick and not quite in time. Every move timid, because they’re still figuring out exactly how this is supposed to work.Then her parents came home, and he was caught fondling their only daughter’s nipples, and that was sort of the end of that. Ford’s first touch was… probably not Zaphod Beeblebrox, but… Because when it comes to those kinds of drinking games (which it almost always does) he likes playing to lose. A lot. Not all of it was great (as far as he can remember), very little of it was sweet, and virtually none of it mattered the next morning or a few days later. Between the two of them, Arthur and Ford have touched each other often – carelessly, to say, “Let’s go to the pub now,” or, “Maybe you’ve had enough for one night” (which is never quite true), or, “So this is it, we’re going to die” (which is never true either). Touching is, of course, quite different. This is still so new, and what if Ford moves too fast or Arthur does something exasperating and everything gets messed up? But, of course, it happens, and when it does there’s so little time to think about it. Arthur is backed up against a wall – “Zarquon, Arthur, not everything has to happen in bed” – and his hands are in Ford’s hair and the whole situation feels so… so fluid. Not natural, exactly, because he feels that the word can’t be stretched to include kissing eagerly against a wall with an alien man on a highly improbable spaceship, but everything leading up to this makes a peculiar kind of sense, so this must too. It fits. Like Ford fits against him. Just… does. And then Ford’s hands are in his dressing gown and under his pajama top, and that is an erection against his thigh, and he can’t catch his breath long enough to panic, so he just doesn’t. Ford grins into the kiss, ecstatic that Arthur is okay with this much. He wants to touch, to possess with his fingers, to go everywhere and explore everything he can on Arthur’s body. There doesn’t seem to be any sort of time limit for this. He finds that surprisingly liberating.His mouth slides down to Arthur’s neck and he’s going to leave a mark there, just above the collar, because he can. The hands in his hair tighten as Arthur gasps – completely unlike any sound Ford has ever head him make before. Do that again, do that again, and he does, and he does.  It’s been a while since either of them has touched anyone like this. For Arthur, a while is a year and a bit, and he feels almost nostalgic; for Ford, it’s slightly under a month, and he feels almost reborn. A kind of baptism in the corridor. In retrospect, they probably should have made it as far as one of their rooms. But Zaphod’s double-take is epic, and even in retrospect Ford thinks it would have been a shame to miss seeing it. And that’s what happened the first time they touched."
ao3_df.story_text[625] = "Sophie Convinced she’s fated to be a failure by pervasive societal messages Depressed and anxious to the point that leaving the house terrifies her Constant excuses to not talk to people Wears one color and it is grey (like her soul) “I’m eighteen but I’m ninety at heart” RAGE Has to get away from her home so she moves into a terrifying moving castle bc it’s free and at this point why not *everything’s on fire* “This is fine” I KNOW EXACTLY HOW THE WORLD WORKS o wait shit fuck Feelings? What are those? I only know cleaning. Howl Fancy degree that he has to move to another world to use Loves his blood family but just… can’t be around them “Why don’t you have a real job” Cannot handle responsibility Is responsible for things anyway “Howl we have no money” “I’m going to buy a skull” Embodiment of Do It For The Aesthetic ™ also “treat yo self” Just Can’t anymore and has a meltdown over his hair (aka everything else in his life) “Oh no I care for all these people why” Overcharges rich people and undercharges poor people bc the system sucks Last night on earth? Time to get wasted. “I don’t care” he says caringly, while caring deeply. Michael Anxious bab What is happening why this plz help Stresses out over his homework only to find out the papers had gotten mixed up and he was doing the wrong thing “I guess this now”: Better at adulting than the actual adults Loves his gf so much and is so openly happy about it A bit useless in a crisis Tries so hard, gets so far Found family is the best family Calcifer “I’m being exploited!” Complains about everything Bored. Just. So bored. Give him attention. Used to be literal star but things went wrong and now he’s living in a fireplace Eats everything and anything. “Guys I have arms be impressed by my arms” Creates a castle for the Aesthetic that also looks like it’s about to fall apart and frightens people. Oops he cares about these assholes now time to ride or die for them"
ao3_df.story_text[639] = "I struggle to tell this story. It’s not mine to tell, first of all. It’s another’s. Yours, you understand? This is a story about the world’s rebirth, but it is also a story of great loss. Yours. Hers. But you’re hardly in a position to say much. Let me try, for nothing succeeds without trial and failure in the Stillness, this continent bisected which you—and, in a fashion, I—call our home. In the aftermath of Gaewha and Alabaster Tenring’s rending of the world, all things may now be different, if we have the will to change them. I, certainly, know this truth firsthand. Where to start? Where you remember, perhaps, what came before. So, let me try to speak of her. You. I hope you’ll forgive me for reopening this wound. This is the story of a beginning and an ending."
ao3_df.story_text[652] = "Sometimes, Arthur wakes up shaking. For stupid reasons, really, but stupid or not he can’t help it. Once it was because he had a nightmare that his pajamas and dressing gown spontaneously burst into flames, and though he wasn’t burned he couldn’t do anything to stop the last of his Earthly possessions from going up in smoke. Another time he fell into an ocean of lukewarm, hideously over-brewed tea and drowned. Other times he wakes up not even remembering what it was, with only the vague sense of things he has lost and can never hope to replace. Sometimes he wakes up and Ford is there, hesitating near the foot of the bed and muttering something about hearing a noise – he sleeps in the next room over, and state-of-the-art ship with a brand-shiny new Infinite Improbability Drive or not, the walls are about as soundproof as Marvin is cheerful. When that happens, Arthur can never quite bring himself to explain what’s wrong. Sometimes he manages to compose himself enough to assure Ford that he’s fine, although it happens often enough that on those occasions the Betelgeusian never really believes him. More and more often Ford doesn’t leave, and every time he stays Arthur suffers the indignity of crying in front of his friend. As time goes on the nightmares are farther between, but that only makes waking up afterwards – surrounded by dredged up nostalgia and clutching a striped mauve towel pilfered from a Vogon spaceship – even more unpleasant. He isn’t surprised to feel Ford shaking him awake. His friend’s presence is the one comforting thing about going through all this, and Arthur has no idea how to tell Ford that. Until, of course, he’s crying on his friend’s shoulder and suddenly blurts out, “Ford, you’re the one comforting thing about going through all this.” Ford smiles a little in the semi-darkness. “I am?” “Yes.” Arthur sniffs and sits up straight. “Honestly, I don’t know why you put up with me.” “Don’t worry about it,” Ford replies, graciously letting Arthur dry his eyes on his towel. “Will you be all right for the rest of the night?” It’s a familiar question, but Arthur hesitates. “I… Could you stay?” he asks in a small voice. It’s the first time he’s dared. For a moment Ford stares at him unblinkingly. Then he says, “Yeah. Yeah, I can stay if you want me to,” and he stands to let Arthur pull the covers back. He slips in beside the Earthman, and it feels perfectly natural when Arthur lets himself be pulled closer. They sleep, peacefully."
ao3_df.story_text[660] = "Who I would miss most and why By Sam Lerman There are but few people whom I can say I would truly miss if something were to happen to them. Most people have passed my path only briefly, leaving nothing but a small imprint, if even that. Those few people whom I would miss, though, are those who changed my life, influenced the way I look at things and altered how I now see myself in such a way, that I do not think I could ever forget about them. I think the one I mention first should be my brother. Because he's the one who has known me the longest, and he's the one who has always been there for me, and to help me, even when I was sure that he would drown in the mess his own life was making. Because he was probably the best gift I ever got, because he was family, and I just got that, as if it were present, a present for my mum getting maried again, and it was a gift worth more than anything I could ever have imagined. He changed my life just by always being there, by letting me know he would always be there, and that is something I don't think I'll ever be able to thank him enough for. The second one is the one I wish I would not miss. Because although he influenced my life more than I ever could've foreseen when I began dating him, it wasn't in a good way. When I look back now, I don't particularly like who I was back then. I don't like that I hurt people then, I don't like what I did because of him, I don't like what I had already done before him. But he did change me, and now I am glad that he did. Because the person who I am now, well, I like her a lot more than I liked the person I was before him, and that's partly due to him. So, yes, I will miss him. But only what he thought me, and the way he could've thought me those things. I wish I hadn't had to learn it that way. The third and last person, is the one I think I would miss more than any other. And it's strange, really, because he would have been - not even a year ago - the person I would have thought of last if someone were to ask me that question. Now though, he is the person I would go to if I had a question, the person I would rely on if I needed something, and the person I would trust if it would be necessary. Because he's the only one who can calm down the chaos that's in my head sometimes. And he's the only one who can make that chaos sort into order. Because he's the only one who can make sitting in a library seem like actual fun. And he's the only one who actually makes it exactly that - fun. Because he's the one who respected me so much, liked me so much, that he was willing to be unhappy for me. And he probably actually was, for some time. Because he's the one who got me to get here, and he's the one who played drag just because one of the actors let us down again, and he's the one who defended my brother when no one else could or would, and he's the one who's been through so much already that it actually manages to give me hope, and he's the one who is always there. And because he might just be the one. Who knows. I once heard somebody say that you should count yourself lucky if you can fit your enemy's on one hand, even luckier if you can count your true friends on the other hand, and absolutely blessed if the hand with your friends holds more weight than the hand with your enemies. I guess that would make me the luckiest person in the world, because I can do all that - and more. I can miss, and even appreciate my 'enemy' for what he once was, or could have been to me. And I have got two of the greatest things one could possibly have, and I don't have to miss either of them."
ao3_df.story_text[680] = "Saf had known that what they had was too good to be true. The quiet days, punctuated with Teddy’s easy laughs and witty comments, and the (surprisingly) tranquil nights when they weren’t out “reclaiming” the endless list of people’s belongings had lulled him into a sort of trance. A trance which made him believe that they could be something resembling forever. He had broken out of it the night they met Sparks for the first time. When Teddy had smiled so readily at her and told her about his “book-of-books,” something that he had hesitated to tell even Saf about, Saf had felt an unfamiliar tightening in his stomach. They’d fought for real that night, hurling words intended to hurt its recipient as much as it hurt its sayer. Saf slept on the cold floor of their room, still smarting from their spat. Come morning, Teddy turned away the moment Saf opened his mouth, and that was when Saf felt the cold edge of dread curling in his belly. Had Teddy finally decided he could do without Saf? -------------- When they’d met Sparks that night, Teddy had seen something in Saf snap, prompting the taller boy to grab his collar and hoist him up against the table. He could feel Sparks’ frightened gaze, darting between Saf’s hand on his collar and his clenched fist hovering just above the table. He knew Saf could have killed him in that second, his breath coming fast and angry for a reason Teddy didn’t know. Teddy is the one who sleeps on the floor that night, making sure that Saf is asleep before allowing his tears to run freely. --------------- Saf’s heart jumps into his throat when he sees the knife flash, quick in quick out of Teddy’s (soft, almost impossibly so) abdomen. He dashes forward, supporting Teddy’s abruptly limp body, and a small, twisted part of him revels in once again holding Teddy close. Once Teddy was hauled back in, the extent of his injury becomes painfully clear. Saf’s stomach twists with fear as he orders Sparks to fetch their healer, and when she’s gone, he sinks down to the ground. His sister shakes his arm, questions overflowing, but he can’t bring himself to speak, afraid that his voice will betray how affected he is. After the healer leaves, Saf lies on his side, watching Teddy’s chest rise and fall. He’s still awake when Teddy stirs the next morning, the numbing effect of the drugs having worn off. He jumps when he feels Teddy’s hand brush against his own, and as Teddy’s fingers move to entwine with his, starts to babble. “Are you in pain? D-Do you need Tilda? I’ll go fetch her at once!” Before he can move, Teddy snatches his wrist and holds on doggedly, grip as unyielding as Saf remembers. His mouth has curved up into a small smile, but his eyes are sad. “‘M sorry,” he mumbles, “for making you worry. Shouldn’t ‘ve done that.” Saf feels his face relax and broaden into a wide grin despite Teddy’s obviously-still-upset face. He tamps down on the urge to pin Teddy to the bed and squash him until he passed out, and opts to retake Teddy’s hand, twining their fingers together. He maneuvers them to lie side-by-side, and this is how Tilda and Bren find them in mid-afternoon, when, confused by the silence, they poke their heads in. ----------- When Teddy has recovered sufficiently from his wound to start pottering about the shop, Saf presses him against the back wall and kisses him. Teddy laughs against his lips. “Missed me?” With those two words, the dam inside Saf breaks and the fear, sadness and anger of the past few weeks tumbles out. “You have no idea how much I worried about you - I thought you were going to die, how dare yo-” Teddy presses his lips to Saf, effectively muffling the rest of his rant. He kisses Saf slowly, coaxing his mouth open and twining their tongues together. When he pulls away, Saf is flushed and panting slightly. “Why would I want to leave this world when you’re still here? Saf, you know I’d never abandon you; I’ve told you that a thousand times! When will you start believing me?” Saf snorts, twisting his face away. “It sure didn’t seem like it when you and Sparks were together. Seemed like you’d follow her to the ends of the world without a second thought.” “But the one I love is you, not her, Saf!” When Saf keeps his eyes fixed somewhere in the vicinity of Teddy’s chin, Teddy huffs. “Why can’t you, for once, understand that I’m not interested in her? Saf, I’m gay. I like boys, not girls. And you're the only person I would ever consider being with, so you can stop that pouting right this instant and get over your stupid mindset about you not being good enough, because you are, and you make me so, so happy.” Teddy cackles when Saf’s face, previously contorted at the mention of Sparks, smooths out into a self-satisfied smile. Saf sniffs at his gleeful look and proceeds to pick Teddy up (carefully, of course - Teddy’s still recovering) and shuffles (being injured didn’t make him any lighter) into their shared bedroom. He dumps Teddy unceremoniously on the bed and then squawks indignantly as Teddy tugs him down onto the mattress and wraps his arms and legs around him, nuzzling into his neck. Saf huffs out a little moan, because of course Teddy would exploit his weak spot when he has to take Teddy’s injury into account and not do some unspeakable things to him like he so desperately wants to do. In order to deflect Teddy’s attention, he digs his fingers into Teddy’s sides, making the boy jerk in surprise. The corner of Saf’s mouth curls up, because of course Teddy would be ticklish; how did he not know this before? Teddy predictably fights back, attacking Saf’s ribs, and the quiet room rapidly descends into chaos. When they’ve both been tickled out of their wits, they lie side by side, both on their backs and panting heavily. Teddy’s eyes are closed, but Saf is frowning heavily at the ceiling, brows drawn together in an intimidating glare. When Teddy notices, he nudges Saf and asks, “Penny for your thoughts?” Saf shrieks and falls off the bed. After Teddy has stopped laughing and the room is silent again, Saf mutters, “I think I love you.” Teddy snorts. “You think? Well I, for one, know that I’m in love with you, shithead.” Saf kicks him, but his chest is full and he can’t stop a wolfish smile from spreading across his face. Teddy pinches him and tells him he looks terrifying, so please can he stop?"
ao3_df.story_text[699] = "this is how they fall in love: (she calls it fate; he calls it an accident but thinks she must be right.) she is wearing a blue dress. her hair is a mess. she is a pretty girl but not overly so; it is the brightness in her eyes that makes her attractive. she is saying, \"i am about to be sick.\" he says, \"take a deep breath.\" she laces her fingertips together, over her diaphragm, and sucks in air as though she is drowning. he puts his hand on her shoulder and says, \"relax, you're going to be okay.\" (at the time he is unaware that he is lying.) -- she sits, knees pulled up to her chest, on the edge of his bed, and stares out the wide expanse of his window. \"hey,\" he says. she startles at the sound of his voice. \"i can't do this,\" she says, shaking. \"i'm going to die.\" he sits next to her, long legs dangling off the edge of the bed. \"no,\" he says, \"you're not.\" (he has never done this before: hence, the optimism.) she sighs and twists around and presses her mouth to his with a quiet desperate exhalation of breath. he startles. \"what--\" \"i'm not going to be a virgin sacrifice,\" she says, slowly, biting her lip. \"i'm not. do this for me-- please?\" he drops his forehead against hers, feeling her shiver against him. he remembers this feeling: the loneliness, pressing against his heart, stifling the beat of it. he fits his hand to the curve of her cheek, and makes it good. -- the darkness hangs across the night sky, so the lamps cast long elegant shadows across the floor and their feet. she slips her hand into his and says, it's dark out, but there are stars, and he drops a kiss on the top of her head and says, yeah. the steady rhythm of her breath settles in his chest, and for the first time in his life he thinks about forever. \"pick one,\" she says, curved up against him, all fragile bones and bare skin-- there isn't enough muscle to take her through a fight. \"a star.\" \"what?\" he asks, distracted by the thump thump thump of her heart against his palm, by the edge of salt caught in the scent of her hair. \"pick a star,\" she says. \"so i can think of you. stars are constant. they don't change.\" he tightens his arms around her and picks the brightest, fiercest star, so that she can know someone is out there, for her. -- she stumbles out of the arena, dripping wet and gasping, eyes flat and hollow. he catches her in his arms and murmurs, oh annie, and she falls into him, breathing out like relief and she says, i saw the star, finn, i saw the star, and throws her exhausted arms around him, too tight for either of them to breathe. the seawater seeps through his expensive, beautiful clothes, so they are the same now, joined in the knowledge that they survived, and the wetness and harsh stink of the sea. the cameras start whirring, and he runs his hands over her hair and says, \"annie cresta, everybody,\" as she presses herself against him. the applause is enough to break his heart."
ao3_df.story_text[701] = "After the Breach is closed, they find themselves far away from the drunken celebrations, locked up safe in Hermann’s room, where Newt is currently kneeling between Hermann’s legs and undoing his belt. “Newton, really,” Hermann says, and god, Newt loves that, that stuffy Victorian decorum even when he’s half-hard, about to get a blowjob. “This is- this is hardly-” “Dude. You want it. I can tell, man, I’m still half in your head, remember?” He tugs Hermann’s cock free from his pants and grins at how it’s already leaking precome. It’s a damn good cock. Newt’s seen, sucked, stroked, and got fucked by plenty of them, and he knows a good cock when he sees one. Well, maybe he’s a little biased in Hermann’s favor. Oh well. Newt licks up the length and feels the electric shiver it sends through Hermann, revels in the way he quakes. He smiles and looks up at Hermann, and the way he’s got his lips pursed together, suppressing a groan, Christ it gets Newt even harder. He decides going slow can wait ‘til another day, a day they’re not celebrating saving the fucking world, and he swallows Hermann’s whole cock in one go. Hermann gasps immediately and grips Newt’s hair, hell yes, grabs it tight like he’s clinging for dear life, and that makes Newt’s own dick throb with interest. He wraps one hand around the base of Hermann’s erection and uses the other to stroke himself, once, through his jeans. He moans, and the sound sends vibrations up Hermann’s shaft. Hermann bucks his hips forward and his cock pushes further into Newt’s mouth- he makes a choking noise, and it burns at the back of his throat- Hermann pulls out instantly, lets go of his hair, and holds Newt’s face in his hands. “Oh, God- Newton, I’m sorry, are you all right? I- I’m sorry, I don’t know what came over me- are you all right?” “No- I mean, yeah, yeah, dude, I’m fine. In fact, uh.” He grins sheepishly up at Hermann. “Could you maybe do that again?” “I- but that must have hurt, I could never-” “It did! Oh, it totally hurt. But, I...” His cock is straining against the seam of his pants, and he’s so achingly hard he could cry. (Not that he would because please, rock stars so don’t cry. Usually. He’ll check that rule again later, after Hermann’s done totally fucking his mouth.) “I really, really wouldn’t mind you doing it again.” Hermann looks hesitant. “Are you sure?” “Positive, man. I swear.” “...All right. I. I will.” Newt grins and fills his mouth again, taking as much of Hermann in as he possibly can. He looks up at Hermann and wiggles his eyebrows, trying to silently say go ahead. Through their connection, he gets a flash of warmth from Hermann, hears a crystal-clear thought, looks so beautiful like this, and his eyes widen with surprise half a second before Hermann thrusts forward again. It hurts, it really does, and he groans and desperately, one-handed, tries to release his own cock from his jeans. He can barely find the zipper, and he gives up and resorts to rubbing himself through the denim. Hermann tightens his fingers in Newt’s hair and thrusts again, and the world goes white. His glasses are falling off his face and he can hardly see, but when he looks up, he can just make out the rapturous look on Hermann’s face. Newt moans and fuck, Hermann just made him come in his jeans like a teenager. “Newton,” Hermann says, sounding strangled. “Newton- Newt, I’m about to-” He gives him a hasty thumbs up and teases at Hermann’s cock with his tongue. Hermann comes down his throat in a searing burn, and Newt does his best not to make a choking noise, because he knows Hermann will get all concerned and embarrassed and he’ll never let Newt do this again. Which would be a goddamn tragedy, honest to god. Hermann pulls out of his mouth and loosens his grip on his hair, and instead of tugging it, he starts petting it, so softly and gently. “Are you all right? Was that- was that fine?” “Dude,” Newt laughs, and his voice is hoarse (Hermann did that, his stuffy, stick-up-his-ass Hermann, how awesome). “I should be asking you that.” Hermann’s cheeks go pink. “It was- it was very good. Truly. Now, I could, ah, return the favor-?” Newt giggles, giddy. “Nah, man. Not, uh, not necessary. Maybe I’ll take you up on that next time.” “Next time,” Hermann repeats, like he’s astounded such a thing could exist. “Quite.”"
ao3_df.story_text[711] = "Perfect white snowflakes fell from the sky, blanketing the world in snow. Peeta wrapped his arms around Katniss’s small frame and held her tight. With a sigh, she hid her face in the crook of his neck, allowing her body to go limp as she leaned into his solid frame. It was getting late. The Christmas dinner was over, and Peeta had to go home, but he wasn’t ready to let go. Not yet. So, he stayed, holding Katniss in his arms for a little bit longer. Gale’s loud laugh floated through the open window, bringing them back to the moment. Reluctantly, Peeta loosened his hold. Katniss pulled away from him. Her stormy gray eyes locked with his, making his heart stutter. The sorrow he found reflected in them made him question every decision he’d made over the last few weeks. Suddenly, his whole life changed. Katniss raised to the tips of her toes and leaned into him once again. Her lips, warm and soft, found his. Surprised, Peeta stood still. Every muscle in his body tensed while his mind raced, desperately trying to process what was happening. Why was his friend’s girlfriend kissing him? His hands settled on her shoulders, gently holding her as he slowly pulled away. His ocean blue eyes narrowed. Anxiously, he searched her face for the answers he needed. “Katniss?” She looked away, her body heavy as she tried to move away from him. He tightened his hold on her. “Talk to me,” he pleaded. Katniss shook her head. Her defeated huff turned into a puffy cloud which flew away, reaching for the darkened winter sky. “I’m sorry,” she mumbled, “I... I just had to do that, at least once.” Ignoring the growing tightness in his chest, he pressed, “You had to... kiss me?” Swallowing loudly, Katniss nodded. “Why?” Warily, she turned to face him once more. She looked scared and sad, and he wished he could just press her against his chest and soothe her worries away. “Because you’re leaving,” she said with a sigh. Her voice dropped down to a whisper, and she added, “And I didn’t want you to go without knowing...” Time stood still in the silence which followed. Peeta held his breath and waited, his heartbeat pounded in his ears, anxiously tracking every passing second as he waited for Katniss to finish her thought. When she didn't speak, he frowned. He knew how uncomfortable words made her, and he didn't want to rush her, but this was too important. He couldn't let her off the hook. Softly, he asked, “Knowing what?” Her chin touched her chest as she mumbled, “That I’m not with Gale anymore.” Peeta gasped. Ice cold wind filled his lungs, startling him. He welcomed the small discomfort, a clear sign that this was not a dream Overwhelmed by an avalanche of conflicting emotions, he kept his eyes trained on Katniss's dejected expression. He didn’t know whether to laugh or cry. Peeta had been in love with Katniss for as long as he could remember. But she had always been completely out of reach. As a teenager, she'd been too busy with her family's problems to think about boys or romance. And recently, she'd started dating one of his best friends. Resigned to being without her, he had decided to go away for a couple of months. He knew he was only fooling himself, but he still hoped the change of scenery in District 4 would help him erase Katniss from his mind. But this, this changed everything. Still stunned, he asked, “What? Since when?” Katniss sighed, her shoulders slumped as she relaxed into Peeta's touch once more. \“We broke up a couple of weeks ago. We haven’t said anything because it’s Christmas and we didn’t want to upset his family, but it’s over.\" Locking her bright silver eyes with his, she added, \"I promise.\” A flash of warm joy rushed through Peeta's body making him forget about the light snowstorm which kept steadily falling over them. Katniss's eyes, bright under the pale moonlight, guided him to her like a lighthouse in a storm. Surprised by the longing and fear he found in them, Peeta wondered whether the flicker of love he now noticed had always been there, and why he’d failed to see it before. His hands, warm and steady cupped her cheeks. With a voice as sweet as golden honey,  he asked, \"Can I kiss you?\" Katniss nodded. Peeta closed his eyes and kissed her, finally getting lost in the feel of her sweet lips on his own. Katniss reached for him, her small hands held on to the soft flannel shirt he wore. Pulling him close to her and whimpering softly as she nibbled on his lower lip. His lungs fought for air and, reluctantly, he broke the kiss. Pressing his forehead against hers, he asked, \"Was this what you had in mind when you kissed me?\" Her flushed cheeks turned even redder as she shyly admitted, \"Yep.\" Peeta chuckled. Her soft admission made his stomach flutter. \"I don't really have to leave now, you know? District 4 can wait.\" Katniss smiled. Her eyes were two bright twinkling stars. \"Really? You'll stay?\" Nodding vigorously, he promised, \"Always.\" Katniss leaned into him once more. Wispy snowflakes hung in the air as they kissed, enjoying the precious gift of each other's love."
ao3_df.story_text[741] = "Alabaster is fond of his doctor. It amazes him, almost amuses him too that he is still capable of growing new fondness. But as He turns up behind the comm leader, it turns out Alabaster is not so dead inside after all. \"You have a personal doctor now.\" She declares, \"Although it will be easier for us all if you save Him the effort.\" He ignores her and eyes the doctor through the infirmary main space. Well hello. The young man looks down at him too, and nods as greeting. They examine each other for a while; He must be assessing his condition (pretty bad, he knows), and him His Everything: not bad at all. \"We plan to stay longer than you like, I'm afraid.\" Finally Alabaster decides to respond properly for the first time, even with a smile, \"We shall work on that together, no?\" The doctor nods again. Then He turns to the comm leader: \"Can I see your medicine stockings?\" He looks like a delicate little bird. \"He is to your liking.\" Antimony throws a plain statement lightly. Alabaster looks up. True. Observant. Thank you for respecting my privacy as always. \"Hmmm.\" Instead he replies. No need. They both know it: nothing good can happen in the shadow of his own creation. The doctor's company benefits Alabaster more than His treatment. He enjoys the texture of Lerna's innocent hands on his skin when He gives him injection, although he can't actually sense it. Doesn't matter. He knows what it would be like. He knew what it would be like. When Lerna is here he secretly scans His face, and when He is off duty, he sesses Him from afar. Is it rude? Sure, but he has done much worse, so fuck ethics and let him have some fun. No wonder Antimony isn't bored stalking him. Observation can be a hobby. The scar on His jaw calls for fondling before his very eyes; those gorgeous Westcoaster lips scream loneliness too. When Lerna takes time to re-examine him, He triggers phantom limb pain on him. Or he just wants to touch Him too badly. His brain isn’t stone yet, for Earth’s sake. Lerna spies his stare and approaches closer to his face. \"Are you alright? Can I give you anything?\" Yes, skinship, please. \"Thank you. You have done enough.\" And then he has to let Him walk away. Lerna's gentleness oddly reminds him of a person he doesn't always want to think of. Their personalities are alike. To be with Him, Alabaster imagines, would feel like finding acceptance too, not to say such wonder could happen. Lerna treats him... normally, which is enough. Kindness, for him, is good enough. He has given up expecting more ages ago and wasn't surprised many times since then. He might try to seduce Lerna, if they were in a different time - a very different one, one that he isn't sure if is possible. He can't have that life. Maybe some other time, and with someone else. Maybe in the future people of his kind can freely put their feelings into action. Maybe. Make rusting haste, Syen; I know you are at your fastest, but our time is running out. Alabaster keeps loosing more of himself to stone. Every bit of his own body becomes more painful, and more... disposable. He can't help being stinky and broken and dying, and, this matters to him, not appealing at all to Lerna; Antimony may be the only person who wants him still. Aware of his own appearance, he wants to avoid Lerna, but can't even manage that. All he could do is to at least feed himself to Antimony while He is away. Sometimes he worries what Syenite would think of him when she arrives. There isn't even much dignity left. Just, a purpose. Hurry, Syen. In the end, I'd rather be with you. Lerna didn't think much of his patient at first. Alabaster doesn't talk, so Lerna leaves Him be. Ykka introduced Him as \"a test for your value\", but Lerna doesn't thinks she really cares whether He's dead or alive. He can't blame her. The man won't last very long with or without him. Too much has happened. He doesn't aim to save every life anymore. For months on the road he even thinks less often of Essun. Chances of meeting her again gets smaller and smaller. Yet recently he thinks of her again, and at times relates her to this dying man who is of her kind and fears whether what hurted Him so heavily could happen to Essun too. He wishes her to be here as well for her safety. But he wasn't waiting, nor expected her to be the person Alabaster is here for. He was happy at first, and then bitter. Lerna learns more about his patient now. That Essun calls him 'Baster. A nickname. They use nicknames. She calls Him an ass, and she knows how to deal with Him. And that they have had a child. And that they used to be lovers. He still has her company and attention. And that they are working together on something. Something orogeny so none of his business. Something that killed Alabaster and now Essun is going to do it too because He tells her to. Lerna finds himself still an outsider to her, but Alabaster is not. He is part of her. And soon, He will imprint her with His death, and she will never forget. He never liked Alabaster much but now he is almost angry at Him. He'd rather be Him."
ao3_df.story_text[765] = "Below the sewer's surface it's particulate and foul, almost impossible to see through—not that one would generally, probably, want to. Brownness stings Mr. Vandemar's eyes and fills his nose. He has held his breath for so long that if he were given to fancy, he might fancy his lungs full enough to buoy him up above the water, up until he bumps against the ceiling like an overly large and horrible balloon. He breaks the surface, plesiosaur-like. \"Any sign?\" barks Mr. Croup, foot tapping. Vandemar's head shakes, spraying muck. Croup sniffs. \"We aren't getting paid nearly enough for this.\""
ao3_df.story_text[777] = "Chapter 1 after katsa returns from giddons castle so katsa was like : that fucking giddon bitch ,thinks he can change my mind and shit. po was looking at her and smiling ,trying not to crack up and how cute she looks ,so ferocious yet so delicatrly her. then katsa looked at him : -oh so u think this is funny huh? -oh no my lady ..he said while motioning a no with his hand and trying to hide his laughter now she started to smile too. Part3 they were tumbling on the mud ,trying to get a good punch on each other .practicing after sundown was a really good idea. po tried to also close his eyes in the beginning for added challenge than quickly realised his mistake when he almost headbutted a tree. Thats when katsa stopped for a fraction of time : -oh so you think im not enough as a challenge you also need to close your eyes?she was amused by this notion -oh i wouldnt dare ..forgive me for trying new things...katsa felt the mockery in his voice and made him trip by putting her leg in front of him,making him loose his balance. what she didnt expect was him clinging onto her sleeves at the last minute to slow his falling . when they both hit the soil with a loud thud po's eyes got big . -im sos orry katsa ..i panicked and didnt realise what i was doing -bitch im fine ,you take care of yourself first ,is your head ok? ,i think there was a log there Whith this po had suddenly felt a sharp pain at the back of his head .he put his hand to check if his head was still intact. katsa's face was of worry while she also tryed to see if his head was bleeding -we should start a fire to see our surrounding and bring some cold water for your head and hope that its just as good as ice.said katsa -you rest here ,il arrange everything .she said ,stepped up and sprinting toward the river nearby. po looked after her .his headacke was already getting better ,meaning it was probably nothing serious .not gonna lie he was enjoying the attention.But he couldnt continue with this so when she came back he assured her he was fine. -are you sure ?she asked while trying to gently clean the mud out of his hair. -yes katsa ,this fairy just healed me with her hands ,he said while turning his head to hold her hands . she actually blushed and thanked the night for concealing it . -she cleared her throught and atteped to stand up -ah it seams like that log wasnt that hard(or maybe too hard ,she murmured )..i should start that fire. he stopped her by continuing to hold her hand ,pulling her back. he just stared at her ,with so much happiness in his eyes -can we stay here just a little bit more ?please? she couldnt object to such geniune ,childish demand. -fine ..dont complain when its cold later .she couldnt stop smiling .her eyes got accustomed to the darkness at this point ,so she knew he was looking between her eyes and lips. she had this urge to laugh so she said in a bubbly voice -what? he answered by leaning close to her and closing his eyes. she closed the distance .the first thing she felt was the little soil particle on his lip that he probably forgot to brush off.she shifted her lips ontp his chin while brushing of that durt his her fingers. meanwhile he leaned his head towards her hair and was engrissed by the earthy smell of her hair."
ao3_df.story_text[810] = "There a calling that rouses me.  I'm never truly asleep, but this crept up on me like a growing storm.  I can feel tide winds in the air. No, go back. Tide currents and forest, go back. There are fate strings here, so many, I barely dare to breathe much less move. Who are these children? They are three, with only the clothes on their backs and whatever is in their pockets and the courage in their hearts. So close to resembling runaways, it triggered my calling. Brought me to them. But I can see more.  There is Destiny with his book, and there are the Fates, and then there is the choices we make.  Fee will is the gift of humankind and it's what makes and breaks prophecies. I can see the golden halo around the three, that makes me halt and step back.  I sniff. The curly haired one, yes, a satyr, a faun, a child of the wild.  Definitely outside my realm. I look at his strings and sigh, but not all who wander are lost and he'll find what he's looking for, even if he doesn't like the ending. The other two, however... if it weren't for the halos, if it weren't for all the strings, blaring red with heart-blood, dripping with possibilities...oh my children, I would weep and hold you close. Maybe call enough power to myself to make sure you get home safely, or dare see whether I can call one of the Guardians; have they crossed from childhood to adulthood? You can never really tell with teens...but with demigods... The girl smells like olive trees, I see owl wings in her hair.  This is wisdom's child, a daughter of Athena, a baby born from wit and intelligence. The boy smells like storms, of quaking earth and crashing tsunamis. I see the waves tossing in his hair, his father's smile. I know that kind of smile. Sassy child. So full of spirit. Sitting back, I look back at the two demigods. They can't see me, can't hear me.  They're sleeping, and all their strings have in common is that they need their rest.  No, not even the satyr can sense me. \"I have small hands,\" I say to them.  They cannot hear me.  \"I have no name, no way to help.\" I know of the wars brewing, the one in the West, and the ones between all the Gods. I am small, too small, to be noticed by any except be absorbed. They are demigods, they have their parents, in theory, to aid them.  Not my realm.  There are rules. I should go.  But I hesitate. I cannot see a clear path for their future; that is up to them, their wits and courage and hearts.  But their pasts... I gently take the strains, now dull with certainty, though no less impacting, still throbbing into the present and future, and I just...skim...a little. I see the child, the children, the ones I couldn't reach, the ones I couldn't lead home, to safety, to protect while at home.  The lost demigods.  And I ache.  But they are beyond me, I cannot help them now. These two, these precious two. No, I glance at the satyr. There is power is the wild things. I cannot help the demigods that have gone before, but maybe I can help the ones to come.\"I will help all that seek my aid,\" I tell them. \"You are all my children.\" I dare to lean over, to kiss each on the forehead, giving them my blessing. \"May you return home.\"  I stop at the boy, Poseidon's son. I can taste the potential in the air. Water is the element of change, I think, and then give him my blessing, \"and may you bring others home as well.\" I leave them then.  There is nothing more to do.  I am a small god, a new god, nameless.  But I was called, and with my small hands, I answered."
ao3_df.story_text[812] = "The janitor working the morning shift found the judge as he was, suspended on chains attached to the courtroom ceiling. The judge’s eyes had been blindfolded, his right hand armed with a sword, his left, with a set of brass scales. On one scale lay the judge’s dissevered heart. On the other lay his brain. The night before, Hannibal Lecter received a visitor at his home. “That noise is vulgar,” he greeted her contemptuously. The woman had been whistling the horn intro to “Minnie the Moocher” as she teetered backward lazily in one of Hannibal’s expensive dining room chairs and appeared to contemplate the portrait of Leda and the Swan across from her. She ceased whistling abruptly and let the front legs of the chair fall to the floor with a sharp cracking noise. “You should find a sweet, tender little Dickensian angel to play your ridiculous harpsichord for you.” Her voice was crisp, frank, and husky, like that of an Old Hollywood actress. “I know how you gobble those ones up.” She stood and faced her host. Her dark, coarse, Roman features had not changed since their last meeting, an age ago. This was with the exception of a modern floor-length toga dress and, to Hannibal’s chagrin, elaborate sleeves of faded green tattoo ink. Her left bottom wrist bore the image of a set of scales; her right, that of a sword. Any bystander would say the woman’s wide, black eyes looked directly into his as she spoke again, but Hannibal knew this was only an exercise of her sharp hearing and sense of direction. “I don’t care for your taste in art, Lecter.” “How do you form opinions on art you can’t see?” he asked, feigning real curiosity. She only smiled dryly at his impertinence. “In any case, shouldn’t you of all people prefer this version of events?” he continued, gesturing toward the painting with a slight tilt of his head. Her smile disappeared. “I’m interested in truth, not artistic license,” she said. “What is truth?” “You’re recycling old lines, dear.” “What are you doing in my home, Dalia?” “What are you doing with my emissary, Lecter?” “Your emissary? Is that what Will is?” Hannibal suppressed a laugh. “Yes.” she replied curtly. “And he’s being tried for your murders.” Hannibal grew serious as he said, “I never meant for harm to come to Will.” “It’s funny how often harm comes to your playthings, Lecter, especially when you don’t mean for it to,” her voice picked up momentum and volume as it went. “You have your share of scores to settle, so I usually leave you to it, with obvious exceptions, namely, when you take one of mine. This conversation should sound familiar to you; we’ve had it on several occasions; why is it that we keep having this conversation, Lecter?” Tears welled in Hannibal’s eyes. “Please,” he whispered. “Please help Will.” “Why should I? You won’t stop with him.” “I’ll do anything.” “You'll never be satisfied.” “I know. Please.” She took Hannibal by the shoulders and slammed him backward into the wall. “There needs to be balance,” she said. “He’ll betray you like you betrayed him. He'll cripple you.” Hannibal nodded frantically, his eyes still pleading. She tore off Hannibal’s jacket, then ripped apart the buttons of his shirt before removing it just as savagely. She slid her hands down Hannibal’s bare shoulders and along his arms until her fingers interlocked with his, lifting their arms in unison until their bodies took on the stance of crucifixion. As she aligned her tattooed wrists with his bare ones, Hannibal felt a slicing pain along the length of his forearms. She muffled his outcry by pressing her lips against his and sinking her teeth into his hubristic tongue. Satisfied, the woman withdrew from Hannibal’s mouth, licking blood from her lips and allowing her new confederate to take a strangled gasp of air. “You owe me an offering,” she smiled into his ear as she released him. She sang placidly to herself as she left, Poor Min, poor Min, poor Min."
ao3_df.story_text[826] = "It was the last night before the first day of the new year, and Ford and Arthur were in a car headed back to Cottington, bickering. “That was completely uncalled for! And you got us thrown out of the party.” “It seemed called for to me.” “She was asking for my number, Ford!” “Exactly.” “Exact— Do you have any idea how long it’s been since I’ve been on a date?” Ford considered this for a moment. “You know, in the four or five years I’ve known you, I don’t know that you’ve been on any dates at all.” “Exactly! I used to. Not a lot of them, but a reasonable amount. Things were going all right for me. Then I met you.” “Me?” Ford asked, hurt. “What did I do?” Arthur wasn’t looking at him. Ostensively because he was concentrating on driving the car through the lightly falling snow, but from the way he was fuming he probably wouldn’t have anyway. “What did you do? You pretended to be far drunker than you were and dumped brandy all down that poor girl’s blouse! Where you got the brandy I’ll never know, because the only drinks at the party were sparkling cider and champagne – but you’re always doing things like that, aren’t you!” “Hold on, now. There may have been a few instances of clumsiness, but hardly—” “I’m not even going to try to remember all the examples I could give you,” Arthur interrupted. “For god’s sake, Ford, are you always this odd or are you just trying to keep women away from me?” At this, Ford blushed brightly. “No,” he said, fiddling with the strap of his satchel, “of course not.” As soon as he said it, he wished he could take it back. He wished he could go back fourteen years and stop himself from hitching a lift with those teasers, just to avoid being there, at that particular moment, telling that particular lie. The car veered to the side of the road suddenly, causing Ford to yelp in alarm. He did so a second time, just for good measure, when they jerked to a halt just as suddenly. Arthur slammed the car into park and turned to stare cryptically at him. “Dingo’s kidneys,” Ford breathed. “What are you trying to do, kill us?!” “Don’t try to change the subject. Were you not telling the truth just now?” Ford tried his best to gather himself and look insulted, but in fact only succeeded in looking shifty. “Whatever gave you the idea that I wasn’t?” “You’re evading the question. And blushing. And I might have had a girl to kiss at midnight if it weren’t for you. I think I’m well within my rights to ask.” It took Ford exactly forty-two seconds to decide to give up and just go for it. (If either of them had thought to count those seconds and remember the number, it may have explained some things a few months later. But then again, it might not have. One never knows.) Surprisingly, Arthur didn’t react badly to the sudden, rather lip-mashing kiss at all. A few moments later, he merely commented, “I thought as much.” “Oh?” Ford asked, feeling rather giddy. “Well, it does explain a lot.” “Oh.” The Betelgeusian smiled a little self-consciously, thinking of how many things about him that Arthur didn’t know might elicit that same response. “I didn’t think I should say anything… “I’ve noticed that some hu – some people don’t go for this sort of thing. I couldn’t tell if you were one of those or not.” Arthur shrugged, and apparently didn’t notice the near slip. “I usually don’t, but why not? You seemed pretty dead set on it and you aren’t a completely unlikable person. I’m just glad we’ve sorted things out before anything more drastic than staining a dry-clean only blouse happened.” “But… weren’t you angry at me earlier?” “Not so much angry as exasperated.” Arthur shifted in his seat and pointed at the radio clock. “Look, it’s three past midnight. What’s your New Year’s resolution, Ford?” After a moment he replied, “Nothing, I suppose. You already have my number.” But secretly he thought, My New Year’s resolution, Arthur Dent, is to take you with me when I finally get a ride off this backwater little planet."
ao3_df.story_text[853] = "“Thanks. Let’s do it again sometime.” You exit the elevator with purpose, feel the door whoosh shut behind you, your fake-of-course-its-fake-everythings-fucking-fake-in-the-capitol hair brush lightly against your back. It feels like Capitol plastic, all synthetic-fibers and freshly-grown-fear, and it’s making your whole body tingle in a way that makes you want to throw the hair into the ceiling fan. You slow the tread of your bare feet against coarse carpet and close your eyes for a moment, and chuckling very lightly. Self-satisfaction ebbs slowly throughout your whole body as you recall the look you just left in the elevator, the whattheactualfuck look that overcame Katniss’ face when you took your clothes off (imeanthatsnodifferentthanusualRIGHTgirlno), and the tiny tingles turn into full-on jolts in your belly as the red-of-her-cheeks registers behind your eyelids– Fucking STOP, Johanna. The FUCK. You open your eyes and narrow your eyelids, somewhere between really-fucking-irritated and actually livid, and it’s making your throat hurt. You draw air quickly through your teeth and choke just a little, not realizing you’re gritting your teeth. You feel a jolt of pain pierce your side, forgetting again about the poorly-healed-breaks-in-your-once-cracked ribs(ohgodohgodpleasedontnotagain) –inhale– and how much it hurts to breathe deeply. But you could give less of a fuck about that right now. Exhaustion overcomes you now, all the self-satisfied splendor leaving your body in awhoosh (like a bellows like–) and you slump against the wall. Your skin itches again, and you shift slightly, absolutely fucking loathing the way the ugly hallway wallpaper feels rubbing against your bare back, but you really can’t motivate yourself enough to move. You press your face into your hands and realize you’re shaking, just a little, full of a fire-fury you didn’t even realize you were capable of anymore. It always was ironic, really, feeling fire-fury as a wooden girl. But then, you always knew you were meant to self destruct someday, burn yourself to oblivion. You narrow your eyes and grit your teeth harder, move your lips justalittle, shape-shifting them from grimace-to-snarl. You bite deep into the just-chapped skin of your painted lips, reveling in the feeling of sharp-bone cutting into fleshy skin. Here you are, naked, pressed against the wall in the middle of the hallway on the seventh floor of the Tribute building, and all you can think of is Katniss-fucking-Everdeen, and the color of her cheeks. You don’t even understand it. She’s the most pathetic fucking thing you’ve ever seen in your life, all prettyface to Peeta’s prettywords, stumbling over sentences, choking on clauses and trying to smile!pretty for the cameras like the rest of theohsofuckingdeadheyheyanotherfucktonight dead-eyed tribute Winners! She’s the worst goddamn actor you’ve ever seen. She’s self-centered in too many ways and self-righteous in others and so utterly fucking brainless, and you can tell can (see it in her face whenever she peeks out Capitol windows to the not-world you live in) that she’s been fucking numb to life since childhood. Except she isn’t pretty, not to you. She’s beyond fucking lovely. And you really fucking hate it. Because of all the filling-un-fucking-fulfilling parts of your miserable fucking life, it’s her, this little lost puppy child who has made you the warmest you’ve been in fucking years. Maybe your whole goddamn miserable life. And she doesn’t even know, doesn’t even fucking realize. She’ll never be able to see, and if you have anything to say about it, will never spend enough time with you in the arena to figure it out (Not that she could the girl is as bright as a bag of nails but who are you fucking kidding you wont actually be able to stay away.) Except. Except. No matter how much she loathes it, what a miserable fucking excuse for a rebellion figurehead the girl is. She’ll always start the slow burn in most fucking wonderfulfuckingpainful way you’ve ever felt; always ignite your body, cause your blood to fucking boil in your veins your chest the lights behind your eyes. Because maybe you’re Johanna fucking Mason, and you might be made of wood-beyond-petrified-into-stone, But she’s the girl on fucking fire. And she’ll always make you burn."
ao3_df.story_text[866] = "Arthur Dent’s first kiss happened under a table. He was six years old and conscientiously helping a girl find some feathered thing she’d dropped. Only, he hadn’t been given a satisfactory explanation of what the feathered thing looked like. (Later he would come to suspect that there never had been a feathered thing at all, but that was quite beside the point.) The point was, she was under the table looking with him and suddenly leaned over and gave him a quick peck on the lips, giggled in that slightly embarrassed way little girls do when they want to get away with something they’re not entirely sure they will, and insisted that now they had to play house and he had to be the husband. Ford Prefect’s first kiss was probably not Zaphod Beeblebrox. On the continuum of all the kissing Ford has done over his lifetime, though, that’s as far back as he can remember – partly because of time, partly because of alcohol, and occasionally also partly due to repression. Zaphod stands out because he was probably the first really good kisser Ford had occasion to run into with his lips, and because Ford actually knew his name. He doesn’t remember the exact circumstances, of course, but there was an awful lot of alcohol involved. Arthur and Ford’s first kiss is like neither of those. It isn’t immature and bashful and really just a lead-in to playing house all afternoon, and neither is it obscene and sloppy and really just a lead-in to sex for the sake of sex. There is some alcohol involved, because Arthur would ruin it by trying to talk too much if he weren’t just tipsy enough to forget to, and because Ford generally dislikes being sober on principle. But neither of them is really drunk. They will remember this in the morning. Remember how Arthur looked so sad about his planet being blown up in the reflection of his jynnan tonnyx, and Ford decided to cheer him up by telling silly stories, and Arthur ended up laughing at some of them in spite of himself. Remember how Ford had thought it was so odd to see Arthur smile again because he hadn’t done it in a while, and leaned in for a closer look, and Arthur had acquired a funny why are you staring at my mouth expression on his face. Remember how Ford had met Arthur’s gaze and how then Arthur had acquired a slightly more wistful why aren’t you staring at my mouth expression, and after that it was only natural to lean in. They’ll remember that, okay, maybe Ford was a little bit drunk, because the kiss tasted like several more different kinds of drinks than just a jynnan tonnyx. Arthur, in particular, will remember not really minding. There wasn’t really time to mind, anyway. Because of, you know, the surprise. Of lips. He’s no stranger to kissing, certainly, so he knows what to do and what to expect, but that doesn’t mean he expected to expect it. By morning they’ll forget the grating squeak of Ford scooting his chair closer, and the fumbling of Arthur’s hands before finding Ford’s upper arms and holding on gently. Ford will forget the little moment of surprise when he realized that Arthur was actually kissing back, and quite earnestly at that – because he’s Ford zarking Prefect, why would he be surprised that someone wants to kiss him. Morning is an imperfect way of referring to it, because there is no morning in space. But, still, when they wake up in the morning to find themselves together in Arthur’s bed, under the covers but still fully dressed (if a little rumpled) except for shoes, they’ll remember it’s because Arthur decided to go to sleep for the night and Ford decided to follow him. They’ll remember kissing again, heads sharing a pillow and arms sort of awkwardly placed because there’s really nothing you can do with the arm on the side you’re lying on that’s very comfortable. At some point in the night they’ve moved around a bit. Ford yawns and rolls over and kisses Arthur good morning, and Arthur makes a face because the morning breath is spectacularly bad, and Ford laughs because of course it is, what does he expect? And that’s what happened the first time they kissed."
ao3_df.story_text[918] = "Mussed in the scarlet linen, quivering still from power and worship, Bilquis closes her eyes. The dying light of the candles dances on the spindle of her thighs, on the dark brown of her skin, smooth and firm anew. Later she will rise and get rid of the dress left on the chair by her last worshipper; once more she will turn on the phone given to her by the Tech Boy. She knows there is a debt here, an obligation she will one day have to fulfil… But how could she regret, when the screen -such a small thing, this screen, and yet so powerful- is the faint opening to the Queen she once was, the Queen she now remember being after so long? She had forgotten that feeling: that fullness, that certainty in herself… Now she knows what she is, what is her due; and she knows that whatever the Network asks in return she will give, for the possibility of forgetting again is unbearable. But all those thought are for later. The now belong to the fulfillment and the evanescent flavour of the veneration so hard to gain in the only temple left to her, to this feeling, even if it is but a pale echo of her past cult. Slowly her breath quiets. Bilquis sleeps, Bilquis dreams. The desert is a vibration of heat beyond the gardens and the high columns of the temple. Yet the marbles of Mahram Bilqis are cool under her bare feet. They echo with the whisper of the fountains and the melody strung out on a qanbüs cords, with the gentle bells of her golden jewellery every time she moves. The air is filled the fragrance of incense and myrrh… Here reigns the Queen of the South, servants and courtiers kneeling at her feets. Her beauty and her wisdom are praised by her people, the sharpness of her mind is feared by her enemies; her name travels with the caravans and with it the glory of Sheba. Strangers come from far away to Ma’rib to gaze upon its wealth and splendour, to cross the eight pillars of the pericycle, to enter the shadows of the temple and venerate her. Not every one of them get to leave. Bilquis is a godess, say some. An enchantress born from the djinns who knows magic beyond the knowledge of men, who sometimes devour her lovers. A woman, just a woman, say others: no more and no less powerful than a woman can ever be. To brush her lips is to stand at the open gates of heaven, to touch her skin is a fulfillment. Many men cross the desert for it; even more write songs. In the dream Bilquis knows all of this, knows it is a dream, a honeyed memory from a vanished past. And yet she let her feet lead her along well-known paths to the center of the temple, let her fingers drag across the words of power engraved in the walls. Yet she listen to the clear voice rising and singing for her, accompanied by the qanbüs.Who is she who looks forth as the morning, Beautiful as the moon, Clear as the sun, Awesome as an army with banners? “It is me”, she whispers in the dream, her heart heavy like a stone. “It was me.”"
ao3_df.additionalTags[918] = "Terrible comme des troupes sous leurs bannières by Jainas"

In [51]:
blankStory_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count
425,How Flowers Bloom In The Wake Of Destruction,batthegrinch,,http://archiveofourown.org/works/16135097,Not Rated,No Archive Warnings Apply,F/M,The Book Thief - Markus Zusak,"Liesel Meminger/Max Vandenburg, Past Liesel Me...","Liesel Meminger, Max Vandenburg, Rudy Steiner,...","Romance, Friends to Lovers, Age Difference, Ni...",NaN,NaN,2018-09-29,8450
519,These Boundaries Crossed,Distracted,,http://archiveofourown.org/works/29491929,Teen And Up Audiences,No Archive Warnings Apply,Gen,Leverage,NaN,"Eliot Spencer (Leverage), Parker (Leverage), A...","Eliot Spencer Whump, Hurt/Comfort, Whump, Team...",NaN,NaN,2021-02-16,4431
772,Book Commentary: Neverwhere,mlmcg12,,http://archiveofourown.org/works/28988598,Teen And Up Audiences,Graphic Depictions Of Violence,Gen,Neverwhere - Neil Gaiman,NaN,Richard Mayhew,"Cross-Posted on WordPress, Spoilers",Book Blog Transcripts,NaN,2020-12-10,195


In [52]:
ao3_df = ao3_df.drop(blankStory_df.index)

In [53]:
blankStory_df = ao3_df[ao3_df["story_text"] == '']
blankStory_df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,additionalTags,series,collections,pub_date,word_count


In [54]:
unique_tags_list = ao3_df.additionalTags.unique().tolist()

tag_list = []

for i in unique_tags_list:
    tag = str(i).split(", ")
    tag_list.append(tag)
    
#code from https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
flattened_tag_list = [item for sublist in tag_list for item in sublist]

flattened_tag_list = list(set(flattened_tag_list))

sorted_flattened_tag_list = sorted(flattened_tag_list)

print(sorted_flattened_tag_list)

['"It\'s the ships fault" Ford says', '"Oh Brother Were Art Thou"', '"Screw the drive of infinite probability" Ford says', '#adaptandresist', '(Canonish) Gay Relationship', '(but not only poetry)', '(though movie fans may like it too idk)', '...or is it?', '100 Drabble Challenge', '100 Themes Challenge', '100words', '173rd', '173rd Airborne Brigade', '221B Ficlet', "2nd time I've used that tag (runs away)", '3 Sentence Ficathon', '3 Sentence Fiction', '30 Days of Writing', '5 Things', '5 Times', '5+1 Things', '69th Games', '74th Hunger Games', '75th Hunger Games', 'A Great And Terrible Beauty', 'A Wind in the Door', 'A cold day in Wakanda', 'A+ Parenting', 'AI romance', 'AO3 is down fic', 'AU', 'Abandoned Work - Unfinished and Discontinued', 'Ableism', 'Ableist Language', 'Abolish I.C.E', 'Abolitionism', 'Absconding with Harry verse', 'Absent John Winchester', 'Abuse', 'Abusive Winchester Family', 'Accident', 'Accidental Marriage', 'Accidentally Baby Acquisition', 'Ace Parker', 'Activi

In [55]:
tag_df = pd.DataFrame(sorted_flattened_tag_list,columns=['tags'])
tag_df.to_csv('UniqueTagListv2.csv', index=False)

In [36]:
#ao3_df.to_csv("processed_ao3_data.csv", index=False)